# Modelling demand response: Checks of updated oemof.solph.custom implementation for SinkDSM

# Package Imports and plot settings
Imports:
* Standard imports
* Import the different implementations for demand response components
* Import module `plotting.py` for extracting results and visualization

Plot settings:<br>
* Register matplotlib converters.
* Adjust matplotlib standard settings for graphs.
* Create a directory to store graphs (if it doesn't already exist).

In [ ]:
import pandas as pd
import numpy as np
import os
import pprint
from math import sqrt

import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from pandas.plotting import register_matplotlib_converters

import oemof.solph as solph
#import custom
from oemof.solph import custom
from oemof.network.network import Node

# Import module for plotting (results handling)
import plotting as plt_dsm

In [ ]:
# Determine matplotlib settings
register_matplotlib_converters()

SMALL_SIZE = 11
MEDIUM_SIZE = 14
BIGGER_SIZE = 15

plt.rc('font', size=SMALL_SIZE)          # controls default text sizes
plt.rc('axes', titlesize=BIGGER_SIZE)    # fontsize of the axes title
plt.rc('axes', labelsize=MEDIUM_SIZE)    # fontsize of the x and y labels
plt.rc('xtick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('ytick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('legend', fontsize=SMALL_SIZE)    # legend fontsize
plt.rc('figure', titlesize=BIGGER_SIZE)  # fontsize of the figure title

In [ ]:
## base dataset
plt_dsm.make_directory('graphics')

# Parameter settings
* General parameter settings for controlling the notebooks workflow
* Special parameter settings for DSM parameterization

## General parameter settings
Control **workflow** of notebook:
* *save_figs*: If True, all figures will be saved in the graphics folder.
* *save_results*: If True, overall amounts of demand response activations will be saved to .csv files
* *write_lp_files*: If True, .lp-Files will be written
* *invest*: If True, investment units for DSM are modeled; if False, dispatch units
* *force_min_invest*: If True, minimum investments in DSM will be forced
* *add_existing_cap*: If Ture, an existing capacity of 100 MW will be given for DSM units

In [ ]:
# Parameters to control overall workflow
save_figs = False
save_results = False
write_lp_files = False
invest = False
force_min_invest = True
add_existing_cap = True

## DSM parameter settings
Define major **parameters concerning demand response modelling**
* *aproaches*: List of the approaches used for demand response modelling; the _interval_ method of the oemof SinkDSM component is named _oemof_ in the following.
* *addition*: Boolean parameter indicating, whether or not to include an additional "logic" constraint into the other DSM modelling approaches that is similar to equation 10 of Zerrahn & Schill (2015)
* *efficiency*: Consider a pontential efficiency loss for those modelling approaches which depict DSM efficiency.
* *recovery_time*: Consider a pontential recovery time for those modelling approaches which depict it (only Zerrahn & Schill 2015).
* *ActivateYearLimit*: Boolean variable indicating whether or not to use a limit for DSM activations per year (only applicable for Gils 2015).
* *ActivateDayLimit*: Boolean variable indicating whether or not to use a limit for DSM activations per day resp. per rolling window (only applicable for Gils 2015).

Determine **costs for demand response**:
* *include_costs*: If True, (small) variable costs will be included.
* *cost_dsm*: Overall variable costs for demand response which have to be splitted up to up and downwards shifts
* *cost_dsm_up*: Costs for upwards shifts (_defaults to have of the overall costs_)
* *cost_dsm_down*: Costs for downwards shifts (_defaults to have of the overall costs_)

Introduce special control variables for controlling the **workflow**, especially for approach from DLR (Gils 2015):
* *introduce_second_dsm_unit*: If True, a second demand response unit with the same parameterization will be introduced.
* *few_timesteps*: If True, for the simple example the timeset will be limited to 9 timesteps in order to increase readability of the .lp-files

In [ ]:
# Parameters focussing on demand response modelling
approaches = ['oemof', 'DIW', 'DLR']
addition = False
efficiency = 1
recovery_time = None
ActivateYearLimit = False
ActivateDayLimit = False

# Determine cost consideration
include_costs = True

if include_costs:
    cost_dsm = 0.1

else:
    cost_dsm = 0

# Cost is split half on upwards and downwards shift; shedding gets high costs
cost_dsm_up = cost_dsm/2
cost_dsm_down_shift = cost_dsm/2
cost_dsm_down_shed = 1000 * cost_dsm

In [ ]:
# Control variables for demand response modelling
introduce_second_dsm_unit = False
few_timesteps = True

# Tools for setup, results extraction and visualization of a (toy) energy system model
For the testing, a **toy energy system** is set up including:
- Coal PP
- Wind PP
- PV PP
- DSM Sink
- shortage
- excess

**Rules for DSM parametrization**:

The following rules apply for parameters which are not part of every modelling approach:<br>
* shift (resp. interference) times: These will be defined half of the delay time and symmetrical in the first place.
* additional parameters: These will be defined to be not bounding in the first place.
* optional parameters & constraints: These will be ignored in the first place.
* additional constraints: These will also be ignored in the first place.

## Create and solve the model
A function is defined here for setting up and solving the toy model.

In [ ]:
def create_model(data, datetimeindex, directory, project, approach,
                 delay_time, shed_time, cost_dsm_up, cost_dsm_down_shift, 
                 cost_dsm_down_shed, efficiency,
                 shed_eligibility, shift_eligibility, introduce_second_dsm_unit,
                 **kwargs):
    """ Function to create and solve the model. """
    
    write_lp_files = kwargs.get("write_lp_files", False)
    
    # Determine whether or not to run an investment model
    invest = kwargs.get('invest', False)
    
    # Control generation units
    include_coal = kwargs.get('include_coal', True)
    include_gas = kwargs.get('include_gas', False)
    nom_cap_coal = kwargs.get('nom_cap_coal', 10000)
    nom_cap_gas = kwargs.get('nom_cap_gas', 10000)
    nom_cap_pv = kwargs.get('nom_cap_pv', 1)


    recovery_time_shift = kwargs.get('recovery_time_shift', None)
    recovery_time_shed = kwargs.get('recovery_time_shed', 24)
    shift_time = kwargs.get('shift_time', delay_time/2)  
    addition = kwargs.get('addition', False)
    fixes = kwargs.get('fixes', False)
    ActivateYearLimit = kwargs.get('ActivateYearLimit', False)
    ActivateDayLimit = kwargs.get('ActivateDayLimit', False)
    
    # Investment modeling
    max_demand = kwargs.get('max_demand', None)
    max_capacity_down = kwargs.get('max_capacity_down', None)
    max_capacity_up = kwargs.get('max_capacity_up', None)
    flex_share_down = kwargs.get('flex_share_down', None)
    flex_share_up = kwargs.get('flex_share_up', None)
    ep_costs = kwargs.get('ep_costs', 1000)
    minimum = kwargs.get('minimum', 0)
    maximum = kwargs.get('maximum', 200)
    existing = kwargs.get('existing', 0)

    # ----------------- Energy System ----------------------------
    
    # Create Energy System
    es = solph.EnergySystem(timeindex=datetimeindex)
                           #groupings=[type])
    len_data = len(data.loc[datetimeindex,:])    
    
    Node.registry = es

    # Create Buses
    if include_coal:
        b_coal_1 = solph.Bus(label='bus_coal_1')
    if include_gas:
        b_gas_1 = solph.Bus(label='bus_gas_1')
    b_elec = solph.Bus(label='bus_elec')

    # Create Sources
    if include_coal:
        s_coal_p1 = solph.Source(label='source_coal_p1',
                                 outputs={
                                    b_coal_1: solph.Flow(
                                        nominal_value=nom_cap_coal,
                                        variable_costs=13)}
                                 )
    if include_gas:
        s_gas_p1 = solph.Source(label='source_gas_p1',
                                 outputs={
                                    b_gas_1: solph.Flow(
                                        nominal_value=nom_cap_gas,
                                        variable_costs=25)}
                                 )        

    s_wind = solph.Source(label='wind',
                          outputs={
                              b_elec: solph.Flow(
                                  fix=data['wind'][datetimeindex],
                                  nominal_value=1)}
                          )

    s_pv = solph.Source(label='pv',
                        outputs={
                            b_elec: solph.Flow(
                                fix=data['pv'][datetimeindex],
                                nominal_value=nom_cap_pv)}
                        )

    # Create Transformers
    if include_coal:
        cfp_1 = solph.Transformer(label='pp_coal_1',
                                  inputs={b_coal_1: solph.Flow()},
                                  outputs={
                                      b_elec: solph.Flow(
                                          variable_costs=0)},
                                  conversion_factors={b_elec: 0.4}
                                  )
    if include_gas:
        gfp_1 = solph.Transformer(label='pp_gas_1',
                                  inputs={b_gas_1: solph.Flow()},
                                  outputs={
                                      b_elec: solph.Flow(
                                          variable_costs=0)},
                                  conversion_factors={b_elec: 0.4}
                                  )
    
    # Create DSM units
    
    # Define kwargs that differ dependent on approach chosen   
    if invest:
        max_demand = None
        max_capacity_down = None
        max_capacity_up = None
    else:
        max_demand = data['demand_el'][datetimeindex].max()
        max_capacity_down = data['Cap_do'][datetimeindex].max()
        max_capacity_up = data['Cap_up'][datetimeindex].max()

    # Define kwargs that are identical for all DSM units
    kwargs_all = {'label': 'demand_dsm',
                  'inputs': {b_elec: solph.Flow(variable_costs=0)},
                  'demand': data['demand_el'][datetimeindex],
                  'capacity_up': data['Cap_up'][datetimeindex],
                  'capacity_down': data['Cap_do'][datetimeindex],
                  'delay_time': delay_time,
                  'shed_time': shed_time,
                  'recovery_time_shift': recovery_time_shift,
                  'recovery_time_shed': recovery_time_shed,
                  'cost_dsm_up': cost_dsm_up,
                  'cost_dsm_down_shift': cost_dsm_down_shift,
                  'cost_dsm_down_shed': cost_dsm_down_shed,
                  'efficiency': efficiency,
                  'shed_eligibility': shed_eligibility,
                  'shift_eligibility': shift_eligibility,
                  'max_demand': max_demand,
                  'max_capacity_down': max_capacity_down,
                  'max_capacity_up': max_capacity_up,
                  'flex_share_down': flex_share_down,
                  'flex_share_up': flex_share_up,
                  'shift_time': shift_time}
    
    # Determine recovery / max activations / cumulative shift / shed time dependent on each other
    if recovery_time_shift is not None:
        n_yearLimit_shift = kwargs.get('n_yearLimit_shift', 
                                       len_data // (delay_time + recovery_time_shift))
        daily_frequency_shift = kwargs.get('daily_frequency_shift', 
                                           24 // (delay_time + recovery_time_shift))
    else:
        n_yearLimit_shift = kwargs.get('n_yearLimit_shift', 
                                       len_data // delay_time)
        daily_frequency_shift = kwargs.get('daily_frequency_shift', 
                                           24 // delay_time)
    
    if recovery_time_shed is not None:
        n_yearLimit_shed = kwargs.get('n_yearLimit_shed', 
                                      len_data // (shed_time + recovery_time_shed))
    else:
        n_yearLimit_shed = kwargs.get('n_yearLimit_shed', 
                                      len_data // shed_time)
    
    cumulative_shift_time = n_yearLimit_shift * shift_time
    cumulative_shed_time = n_yearLimit_shed * shed_time
    
    if not daily_frequency_shift == 0:
        t_dayLimit = kwargs.get('t_dayLimit', 
                                24 / daily_frequency_shift - 1)
    else:
        t_dayLimit = kwargs.get('t_dayLimit', 0) 

    kwargs_dict = { 
        'oemof': {'approach': approach,
                  'shift_interval': kwargs.get('shift_interval', 24)},

        'DIW': {'approach': approach},
        
        'DLR': {'approach': approach,
                'ActivateYearLimit': ActivateYearLimit,
                'ActivateDayLimit': ActivateDayLimit,
                'n_yearLimit_shift': n_yearLimit_shift,
                'n_yearLimit_shed': n_yearLimit_shed,
                't_dayLimit': t_dayLimit,
                'addition': addition,
                'fixes': fixes},
                  }
    
    # Optionally attribute half of the potential to a second identical dsm unit with a new label
    if introduce_second_dsm_unit:
        kwargs_all['demand'] = kwargs_all['demand']/2
        kwargs_all['capacity_up'] = kwargs_all['capacity_up']/2
        kwargs_all['capacity_down'] = kwargs_all['capacity_down']/2
        
        # Copy data and drop label as well as inputs since these cause trouble otherwise
        kwargs_all_manipulated = kwargs_all.copy()
        kwargs_all_manipulated.pop('label')
        kwargs_all_manipulated.pop('inputs')
    
    if invest:
        maximum_wind = data['wind'][datetimeindex].max()
        maximum_demand = kwargs_all['demand'].max()
        max_used_for_normalization = max(maximum_wind, maximum_demand)
    
    # Update some kwargs
    # since they have been changed (i.e. normalized) for investment modeling
    kwargs_all_invest = kwargs_all.copy()
    if not invest:
        kwargs_all_invest['demand'] = kwargs_all['demand'].div(kwargs_all['demand'].max())
    else:
        kwargs_all_invest['demand'] = kwargs_all['demand'].div(max_used_for_normalization)
        
    kwargs_all_invest['capacity_up'] = kwargs_all['capacity_up'].div(kwargs_all['capacity_up'].max())
    kwargs_all_invest['capacity_down'] = kwargs_all['capacity_down'].div(kwargs_all['capacity_down'].max())
    
    # Actually build the units
    if approach in ["DIW", "oemof", "DLR"]:
        
        if not invest:
            demand_dsm = custom.SinkDSM(**kwargs_all_invest,
                                         **kwargs_dict[approach])
        else:
            demand_dsm = custom.SinkDSM(**kwargs_all_invest,
                                         **kwargs_dict[approach],
                                         investment=solph.options.Investment(
                                             existing=existing,
                                             minimum=minimum,
                                             maximum=maximum,
                                             ep_costs=ep_costs)
                                         )
        
        if introduce_second_dsm_unit:
            demand_dsm2 = custom.SinkDSM(label="demand_dsm2",
                                          inputs={b_elec: solph.Flow(variable_costs=0)},
                                          **kwargs_all_manipulated,
                                          **kwargs_dict[approach])

    # Backup excess / shortage
    excess = solph.Sink(label='excess_el',
                        inputs={b_elec: solph.Flow(variable_costs=1)}
                        )

    s_shortage_el = solph.Source(label='shortage_el',
                                 outputs={
                                     b_elec: solph.Flow(
                                         variable_costs=200)}
                                 )

    # -------------------------- Create Model ----------------------
    
    #pprint.pprint(es.groups, width=1)
    
    # Create Model
    model = solph.Model(es)

    # Solve Model
    model.solve(solver='gurobi', solve_kwargs={'tee': False})

    # Write LP File
    if write_lp_files:
        filename = os.path.join(os.path.dirname('__file__'), directory, project +'.lp')
        model.write(filename, io_options={'symbolic_solver_labels': True})

    # Save Results
    es.results['main'] = solph.processing.results(model)
    es.results['meta'] = solph.processing.meta_results(model)
    es.dump(dpath=None, filename=None)

    return model

## Extract model results and plot the model
A function is defined here to extract results from the model and plot the model results.

In [ ]:
def start_model(df_data, timesteps, **kwargs):
    """ Function to extract model results and plot the model. """
    
    invest = kwargs.get('invest', False)
    
    approach = kwargs.get('approach', 'DIW')
    
    # Control plotting and processing
    case = kwargs.get('case', 'constant')
    param_variations = kwargs.get('param_variations', '')
    plot = kwargs.get('plot', False)
    figure_size = kwargs.get('figsize', (15,10))
    show = kwargs.get('show', False)
    save = kwargs.get('save', False),
    write_lp_files = kwargs.get("write_lp_files", False)
    ax1_ylim = kwargs.get('ax1_ylim', [-10, 250])
    ax2_ylim = kwargs.get('ax2_ylim', [-110, 150])
    include_generators = kwargs.get('include_generators', False)
    
    # Control generation units
    gen_dict = {'include_coal': kwargs.get('include_coal', True),
                'include_gas': kwargs.get('include_gas', False),
                'nom_cap_coal': kwargs.get('nom_cap_coal', 10000),
                'nom_cap_gas': kwargs.get('nom_cap_gas', 10000),
                'nom_cap_pv': kwargs.get('nom_cap_pv', 1)}
    
    # ----------------- Input Data & Timesteps ----------------------------

    # Provide directory
    project = 'demand_shift_' + approach + '_' + case + param_variations
    directory = './'

    # Data manipulation
    data = df_data

    # Timestamp
    datetimeindex = pd.date_range(start='1/1/2013',
                                  periods=timesteps,
                                  freq='H')
    
    len_data = len(data.loc[datetimeindex,:])
    
    # ----------------- DSM Parameterization ----------------------------
    
    introduce_second_dsm_unit = kwargs.get('introduce_second_dsm_unit', False)
    
    # kwargs for all approaches
    delay_time = kwargs.get('delay_time', 1)
    shed_time = kwargs.get('shed_time', 1)
    cost_dsm_up = kwargs.get('cost_dsm_up', 0)
    cost_dsm_down_shift = kwargs.get('cost_dsm_down_shift', 0)
    cost_dsm_down_shed = kwargs.get('cost_dsm_down_shed', 0)
    efficiency = kwargs.get('efficiency', 1)
    shed_eligibility = kwargs.get('shed_eligibility', False)
    shift_eligibility = kwargs.get('shift_eligibility', True)
    
    # Investment modeling
    max_demand = data['demand_el'][datetimeindex].max()
    max_capacity_down = data['Cap_do'][datetimeindex].max()
    max_capacity_up = data['Cap_up'][datetimeindex].max()
    flex_share_down = kwargs.get('flex_share_down', None)
    flex_share_up = kwargs.get('flex_share_up', None)
    ep_costs = kwargs.get('ep_costs', 1000)
    minimum = kwargs.get('minimum', 0)
    maximum = kwargs.get('maximum', 200)
    existing = kwargs.get('existing', 0)

    if invest:
        max_demand = None
        max_capacity_down = None
        max_capacity_up = None
    
    # kwargs that differ dependent on approach chosen
    shift_time = kwargs.get('shift_time', delay_time/2)
    
    # determine recovery / max activations / cumulative shift / shed time dependent on each other
    recovery_time_shift = kwargs.get('recovery_time_shift', None)
    recovery_time_shed = kwargs.get('recovery_time_shed', 24)
    
    if recovery_time_shift is not None:
        n_yearLimit_shift = kwargs.get('n_yearLimit_shift', 
                                       len_data // (delay_time + recovery_time_shift))
        daily_frequency_shift = kwargs.get('daily_frequency_shift', 
                                           24 // (delay_time + recovery_time_shift))
    else:
        n_yearLimit_shift = kwargs.get('n_yearLimit_shift', 
                                       len_data // delay_time)
        daily_frequency_shift = kwargs.get('daily_frequency_shift',
                                           24 // delay_time)
   
    if recovery_time_shed is not None:
        n_yearLimit_shed = kwargs.get('n_yearLimit_shed', 
                                      len_data // (shed_time + recovery_time_shed))
    else:
        n_yearLimit_shed = kwargs.get('n_yearLimit_shed', 
                                      len_data // shed_time)
    
    cumulative_shift_time = n_yearLimit_shift * shift_time
    cumulative_shed_time = n_yearLimit_shed * shed_time

    if not daily_frequency_shift == 0:
        t_dayLimit = kwargs.get('t_dayLimit', 
                                24 / daily_frequency_shift - 1)
    else:
        t_dayLimit = kwargs.get('t_dayLimit', 0) 
    
    kwargs_dict = {
        'oemof': {'shift_interval': kwargs.get('shift_interval', None)},

        'DIW': {},
        
        'DLR': {'shift_time': shift_time,
                'ActivateYearLimit': kwargs.get('ActivateYearLimit', False),
                'ActivateDayLimit': kwargs.get('ActivateDayLimit', False),
                'n_yearLimit_shift': n_yearLimit_shift,
                'n_yearLimit_shed': n_yearLimit_shed,
                't_dayLimit': t_dayLimit,
                'addition': kwargs.get('addition', False),
                'fixes': kwargs.get('fixes', False)},            
                  }
    
    # ----------------- Create & Solve Model ----------------------------

    # Create model
    model = create_model(data,
                         datetimeindex, 
                         directory, 
                         project,
                         approach,
                         delay_time, 
                         shed_time,
                         cost_dsm_up, 
                         cost_dsm_down_shift,
                         cost_dsm_down_shed,
                         efficiency,
                         shed_eligibility,
                         shift_eligibility,
                         introduce_second_dsm_unit,
                         recovery_time_shift=recovery_time_shift,
                         recovery_time_shed=recovery_time_shed,
                         invest=invest,
                         max_demand=max_demand,
                         max_capacity_down=max_capacity_down,
                         max_capacity_up=max_capacity_up,
                         flex_share_down=flex_share_down,
                         flex_share_up=flex_share_up,
                         existing=existing,
                         ep_costs=ep_costs,
                         minimum=minimum,
                         maximum=maximum,
                         write_lp_files=write_lp_files,
                         **kwargs_dict[approach],
                         **gen_dict)

    # Get Results
    es = solph.EnergySystem()
    es.restore(dpath=None, filename=None)
    
    results = es.results['main']
    meta = es.results['meta']
    
    # Export data
    if invest:
        df_gesamt, dsm_invest = plt_dsm.extract_results(model, approach, 
                                                        **gen_dict, 
                                                        invest=True,
                                                        max_demand_inv=kwargs.get('max_demand_inv', 1),
                                                        max_capacity_down_inv=kwargs.get('max_capacity_down_inv', 1),
                                                        max_capacity_up_inv=kwargs.get('max_capacity_up_inv', 1))     
    else:
        df_gesamt = plt_dsm.extract_results(model, approach, 
                                            **gen_dict, 
                                            invest=False,
                                            normalized=True,
                                            max_demand=max_demand,
                                            max_capacity_down=max_capacity_down,
                                            max_capacity_up=max_capacity_up)
    
    # ----------------- Plot Results ----------------------------
    if plot:
        plt_dsm.plot_dsm(df_gesamt,
                directory,
                project,
                **kwargs_dict[approach],
                days=2,
                show=show,
                figsize=figure_size,
                ax1_ylim=ax1_ylim,
                ax2_ylim=ax2_ylim,
                include_generators=include_generators,
                approach=approach,
                save=save)

    if invest: 
        return df_gesamt, model, meta, dsm_invest
    else:
        return df_gesamt, model, meta

## Plot case (availability and generation data)
Function to visualize the case considered for the simple example model
* Show availability, i.e. capacity bounds for DSM
* Show demand before DSM and generation pattern as well as "residual load"

In [ ]:
def plot_case(data, case='constant', **kwargs):
    """ Function to plot the case considered.
    
    Case is defined by availability time series, i.e. capacity bounds for DSM and
    demand before DSM as well as generation pattern.
    """
    show = kwargs.get('show', True)
    save_figs = kwargs.get('save_figs', True)
    
    # Plot demand, wind generation and DR capacity limits
    fig = plt.figure(figsize=(15,4))
    ax = fig.add_subplot(111)

    _ = plt.title('Generation and demand for case "' + case + '"')

    # Define xaxis ticks
    ax.xaxis_date()
    ax.xaxis.set_major_formatter(mdates.DateFormatter('%d.%m - %H h'))  # ('%d.%m-%H h'))
    ax.set_xlim(data.index.values[0] - pd.Timedelta(1, 'h'), 
                data.index.values[0] + pd.Timedelta(1, 'h'))
    plt.xticks(pd.date_range(start=data.index.values[0], 
                             periods=len(data)+1, 
                             freq='H'), rotation=90)

    ax.plot(data.index, data['demand_el'].values, drawstyle="steps-post", label="demand")
    ax.plot(data.index, data['wind'].values, drawstyle="steps-post", label="generation")

    # Cap_up and Cap_do only included for proper alignment here
    ax.plot(data.index, (data['demand_el'] + data['Cap_up']).values, 
            drawstyle="steps-post", color="limegreen", label="upper limit")
    ax.plot(data.index, (data['demand_el'] - data['Cap_do']).values, 
            drawstyle="steps-post", color="lightcoral", label="lower limit")
    
    _ = ax.set_yticks(range(-(data.Cap_do.max()-100), data.Cap_up.max()+125, 25))
    ax.legend(bbox_to_anchor=(0., -0.5, 1., 0.102), loc=2, ncol=2, borderaxespad=0.)
    _ = ax.set_xlabel("Time in h")
    _ = ax.set_ylabel("capacity in MW \n(demand, generation,\n abs. limits)")

    plt.grid(alpha=0.6)
    
    # Delta MW on secondary y_axis
    ax2 = ax.twinx()
    ax2.xaxis.set_major_formatter(mdates.DateFormatter('%d.%m - %H h'))  # ('%d.%m-%H h'))
    ax2.set_xlim(data.index.values[0] - pd.Timedelta(1, 'h'), 
                data.index.values[-1] + pd.Timedelta(1, 'h'))
    plt.xticks(pd.date_range(start=data.index.values[0], 
                             periods=len(data)+1, 
                             freq='H'), rotation=90)

    ax2.plot(data.index, data.Cap_up.values, drawstyle="steps-post", #secondary_y=True, 
             linestyle=":", color="darkgreen", label="Cap_up (right axis)")
    ax2.plot(data.index, (data.Cap_do*-1).values, drawstyle="steps-post", #secondary_y=True, 
             linestyle=":", color="saddlebrown", label="Cap_do (right axis)")
    
    _ = ax2.set_yticks(range(-data.Cap_do.max(), data.Cap_up.max()+50, 50))
    ax2.legend(bbox_to_anchor=(0., -0.5, 1., 0.102), loc=1, ncol=1, borderaxespad=0.)
    _ = ax2.set_ylabel("difference $\Delta$ MW \n(Cap_up, Cap_do)")#
    
    # Do axis aligment
    plt_dsm.align_yaxis(ax, -(data.Cap_do.max()-100), ax2, -data.Cap_do.max())
    plt_dsm.align_yaxis(ax, data.Cap_up.max()+100, ax2, data.Cap_up.max())

    if show:
        plt.show()

    if save_figs:
        name = 'toy-model_' + case + '.png'
        fig.savefig('./graphics/' + name)
        plt.close()
        print(name + " saved.")

In [ ]:
def plot_case_residual(data, case='constant', **kwargs):
    """ Function to plot the residual load for the respective case. 
    
    Residual load is defined here as the difference between
    generic generation and demand, i.e., what is actually to be balanced.
    """
    show = kwargs.get('show', True)
    save_figs = kwargs.get('save_figs', True)
    
    fig = plt.figure(figsize=(15,4))
    ax = fig.add_subplot(111)
    _ = plt.title('"Residual load" for case "' + case + '"')

    ax.xaxis_date()
    ax.xaxis.set_major_formatter(mdates.DateFormatter('%d.%m - %H h'))  # ('%d.%m-%H h'))
    ax.set_xlim(data.index.values[0] - pd.Timedelta(1, 'h'), 
                data.index.values[0] + pd.Timedelta(1, 'h'))
    plt.xticks(pd.date_range(start=data.index.values[0], 
                             periods=len(data)+1, 
                             freq='H'), rotation=90)

    ax.plot(data.index, (data['wind'] - data['demand_el']).values, drawstyle="steps-post", 
            linestyle="-.", label="residual load", color="black")
    _ = ax.set_yticks(range(-100,125,25))
    plt.grid()
    _ = ax.set_ylabel("MW \n(residual load)")
    
    if show:
        plt.show()

    if save_figs:
        name = 'toy-model_' + case + '_residual.png'
        fig.savefig('./graphics/' + name)
        plt.close()
        print(name + " saved.")

## Create and display a results table
Function to compare the overall results as far as amount of DSM activations, generation and shortage / excess is concerned.

In [ ]:
def show_results_table(approach_dict, save_results, MultiIndex=False, 
                       param_name="costs", decimals=0):
    """ Show and visualize the results from the previous model run """
    # Show total values for the timeframe considered
    if not MultiIndex:
        dsm = pd.DataFrame()
    else:
        dsm = pd.DataFrame(columns=pd.MultiIndex.from_tuples(approach_dict.keys()))
        dsm.columns.rename(["approach", param_name], inplace=True)
  
    for index, df in approach_dict.items():
        dsm[index]  = df[0].abs().sum().round(decimals=decimals)
        dsm.loc['gen_total', index] = df[0][['wind', 'pv', 'coal1']].sum().sum().round()
        dsm.loc['gen_generic', index] = df[0][['wind', 'pv']].sum().sum().round()

    display(dsm.loc[['demand_el','dsm_tot','excess',
                     'cap_up', 'cap_do',
                     'gen_total','gen_generic',
                     'wind', 'pv', 'coal1']].T.style.bar(axis=0 ,color='goldenrod'))

    if save_results:
        name = 'results_' + case + '.csv'
        dsm.loc[['demand_el','dsm_tot','excess',
                     'cap_up', 'cap_do',
                     'gen_total','gen_generic',
                     'wind', 'pv', 'coal1']].T.to_csv('./graphics/' + name, sep=';', decimal=',')
        print(name + ' saved.')

# Basic comparison for simple setting
In the following, the results for the toy model considerations are depicted. This section is structured as follows:
* At first a base data set is defined, consisting of a flat demand, a fixed generic generation and capacity limits for DSM.
* Then, the demand and/or generic generation the above stated cases are evaluated for the DSM behaviour. The cases are:
    * Case 1: Simple demand variation
    * Case 2: Demand variations
        * 2a: variations with the same amplitude each
        * 2b: variations with a changing amplitude
        * 2c: variations with changed starting timesteps for load shifts
        * 2d: variations with a longer duration (shift for several subsequent hours)
        * 2e: variations demanding for longer shift times
    * Case 3: Increase in demand (use case for load shedding)
    * Case 4: (asymmetric) variations in generation and constant demand
    * Case 4: variations in demand and generation 

## Determine base data set
* A basic data set for the toy model is defined in the following.
* To analyze different behaviour of the modelling approaches, this data set is modified for the different cases.

In [ ]:
timesteps = 48

# base data set
demand = [100] * timesteps
pv = [0] * timesteps
capup = [100] * timesteps
capdo = [100] * timesteps
wind = [100] * timesteps
 
base = [demand, wind, capup, capdo, pv]
df_base = pd.DataFrame(list(zip(*base)))
df_base.rename(columns={0:'demand_el',1:'wind', 2:'Cap_up', 3:'Cap_do', 4:'pv'}, inplace=True)
df_base['timestamp'] = pd.date_range(start='1/1/2013', periods=timesteps, freq='H')
df_base.set_index('timestamp', drop=True, inplace=True)

In [ ]:
plot_case(data=df_base, case="base data set")
plot_case_residual(data=df_base, case="base data set")

## Case 1: Only one demand variation and constant generation
Analyze one of the most simple examples. Slice only a few timesteps to get an easily readable .lp-file.

In [ ]:
case = 'simple'

# Base data set
df_data = df_base.copy()
demand = [100] * timesteps

# Manipulate demand
demand[1:2] = [150]
demand[5:6] = [50]

df_data['demand_el'] = demand
df_data['Cap_up'] = [100] * timesteps + df_data['Cap_up'] - df_data['demand_el']
df_data['Cap_do'] = [100] * timesteps + df_data['demand_el'] - df_data['Cap_do']

In [ ]:
plot_case(data=df_data, case=case)
plot_case_residual(data=df_data, case=case)

In [ ]:
# Slice the first couple of hours to obtain readable .lp files
if few_timesteps:
    df_data = df_data[:9]

In [ ]:
minimum = 0
existing = 0

if invest:
    flex_share_down = 1
    flex_share_up = 1
else:
    flex_share_down = None
    flex_share_up = None

In [ ]:
# Introduce a dict to store the results of every approach
approach_dict = {}

if add_existing_cap:
    existing=100
if force_min_invest:
    minimum = 149 - existing

for approach in approaches:
    print(f'Using approach {approach}')
    approach_dict[approach] = start_model(df_data, timesteps=len(df_data), plot=True, save=save_figs, case=case, 
                                          shift_interval=24, delay_time=4, efficiency=1, approach=approach,
                                          cost_dsm_up=cost_dsm_up, cost_dsm_down_shift=cost_dsm_down_shift, 
                                          cost_dsm_down_shed=cost_dsm_down_shed,
                                          addition=True, recovery_time_shift=None, recovery_time_shed=4, 
                                          figsize=(15,8), 
                                          shed_eligibility=False, shed_time=4, n_yearLimit_shed=6,
                                          introduce_second_dsm_unit=introduce_second_dsm_unit,
                                          invest=invest,
                                          flex_share_down=flex_share_down,
                                          flex_share_up=flex_share_up,
                                          ep_costs=0.001,
                                          existing=existing,
                                          minimum=minimum,
                                          maximum=200,
                                          max_demand_inv=df_data['demand_el'].max(),
                                          max_capacity_down_inv=df_data['demand_el'].max(),
                                          max_capacity_up_inv=200-df_data['demand_el'].min()
                                         )

In [ ]:
show_results_table(approach_dict, save_results)

**Results validation / debugging**

In [ ]:
results_dict = {}
for k, v in approach_dict.items():
    print(k)
    display(v[2]['objective'])
    results_dict[k] = v[2]['objective']
    
results_df = pd.DataFrame.from_dict(results_dict, orient='index')
#display(results_df)

In [ ]:
if invest:
    for k, v in approach_dict.items():
        print(f"investment in DR for approach {k}: {v[3]} MW of installed capacity")

In [ ]:
if save_results:
    if minimum != 0:
        results_df.to_csv('target_vals_' + case + '_min_invest' + '.csv')
    else:
         results_df.to_csv('target_vals_' + case + '_no_min_invest' + '.csv')       

## Case 2: Variation in demand and constant generation

There are several cases in which the demand pattern is altered in order to analyze the DSM behaviour. Compared to variations in generation (which are basically interchangeable to demand variations if they happen in the opposite direction), the benefit is that deviations in demand can be easily identified in the results plots.

### Case 2a: Variations in demand (same amplitude) and constant generation
Substitute flat demand by introducing two demand peaks and drops each per day with a constant amplitude.

In [ ]:
case = 'dem_variation'

# Data preperation: manipulate demand data
df_data = df_base.copy()
demand = [100] * timesteps

# demand changes
demand[4:5] = [150]
demand[5:6] = [50]

demand[11:12] = [150]
demand[13:14] = [50]

if timesteps > 24:
    demand[26:27] = [50]
    demand[29:30] = [150]

    demand[36:37] = [50]
    demand[40:41] = [150]

df_data['demand_el'] = demand
df_data['Cap_up'] = [100] * timesteps + df_data['Cap_up'] - df_data['demand_el']
df_data['Cap_do'] = [100] * timesteps + df_data['demand_el'] - df_data['Cap_do']

In [ ]:
plot_case(data=df_data, case=case)
plot_case_residual(data=df_data, case=case)

In [ ]:
# Introduce a dict to store the results of every approach
approach_dict = {}

minimum = 149 - existing

for approach in approaches:
    approach_dict[approach] = start_model(df_data, timesteps=len(df_data), plot=True, save=save_figs, case=case, 
                                          shift_interval=24, delay_time=4, efficiency=1, approach=approach,
                                          cost_dsm_up=cost_dsm_up, cost_dsm_down_shift=cost_dsm_down_shift, cost_dsm_down_shed=cost_dsm_down_shed,
                                          addition=False, recovery_time_shift=None, recovery_time_shed=4, figsize=(15,8), 
                                          shed_eligibility=True, shed_time=4, n_yearLimit_shed=6,
                                          introduce_second_dsm_unit=introduce_second_dsm_unit,
                                          invest=invest,
                                          flex_share_down=flex_share_down,
                                          flex_share_up=flex_share_up,
                                          ep_costs=0.001,
                                          existing=existing,
                                          minimum=minimum,
                                          maximum=200,
                                          max_demand_inv=df_data['demand_el'].max(),
                                          max_capacity_down_inv=df_data['demand_el'].max(),
                                          max_capacity_up_inv=200-df_data['demand_el'].min(),
                                          ActivateYearLimit=True,
                                          n_yearLimit_shift=1
                                         )

In [ ]:
show_results_table(approach_dict, save_results)

**Results validation / debugging**

In [ ]:
results_dict = {}
for k, v in approach_dict.items():
    print(f"approach: {k}\t objective: {v[2]['objective']}")
    results_dict[k] = v[2]['objective']
    
results_df = pd.DataFrame.from_dict(results_dict, orient='index')

In [ ]:
if invest:
    for k, v in approach_dict.items():
        print(f"investment in DR for approach {k}: {v[3]} MW of installed capacity")

In [ ]:
if save_results:
    if minimum != 0:
        results_df.to_csv('target_vals_' + case + '_min_invest' + '.csv')
    else:
         results_df.to_csv('target_vals_' + case + '_no_min_invest' + '.csv')       

### Case 2b: Variations in demand (changing amplitude) and constant generation
Substitute flat demand by introducing two demand peaks and drops each per day with a changing, i.e. increasing amplitude.

In [ ]:
case = 'dem_variation_amplitude'

# Data preperation: manipulate demand data
df_data = df_base.copy()
demand = [100] * timesteps

# demand changes
demand[4:5] = [125]
demand[5:6] = [75]

demand[11:12] = [150]
demand[13:14] = [50]

if timesteps > 24:
    demand[26:27] = [25]
    demand[29:30] = [175]

    demand[36:37] = [0]
    demand[40:41] = [200]

df_data['demand_el'] = demand
df_data['Cap_up'] = [100] * timesteps + df_data['Cap_up'] - df_data['demand_el']
df_data['Cap_do'] = [100] * timesteps + df_data['demand_el'] - df_data['Cap_do']

In [ ]:
plot_case(data=df_data, case=case)
plot_case_residual(data=df_data, case=case)

In [ ]:
# Introduce a dict to store the results of every approach
approach_dict = {}

minimum = 149 - existing

for approach in approaches:
    approach_dict[approach] = start_model(df_data, timesteps=len(df_data), plot=True, save=save_figs, case=case, 
                                          shift_interval=24, delay_time=4, efficiency=1, approach=approach,
                                          cost_dsm_up=cost_dsm_up, cost_dsm_down_shift=cost_dsm_down_shift, cost_dsm_down_shed=cost_dsm_down_shed,
                                          addition=False, recovery_time_shift=None, recovery_time_shed=4, figsize=(15,8), 
                                          shed_eligibility=True, shed_time=4, n_yearLimit_shed=6,
                                          introduce_second_dsm_unit=introduce_second_dsm_unit,
                                          invest=invest,
                                          flex_share_down=flex_share_down,
                                          flex_share_up=flex_share_up,
                                          existing=existing,
                                          ep_costs=0.001,
                                          minimum=minimum,
                                          maximum=200,
                                          max_demand_inv=df_data['demand_el'].max(),
                                          max_capacity_down_inv=df_data['demand_el'].max(),
                                          max_capacity_up_inv=200-df_data['demand_el'].min(),
                                          ActivateYearLimit=True,
                                          n_yearLimit_shift=1,
                                         )

In [ ]:
show_results_table(approach_dict, save_results)

**Results validation / debugging**

In [ ]:
results_dict = {}
for k, v in approach_dict.items():
    print(f"approach: {k}\t objective: {v[2]['objective']}")
    results_dict[k] = v[2]['objective']
    
results_df = pd.DataFrame.from_dict(results_dict, orient='index')

In [ ]:
if invest:
    for k, v in approach_dict.items():
        print(f"investment in DR for approach {k}: {v[3]} MW of installed capacity")

In [ ]:
if save_results:
    if minimum != 0:
        results_df.to_csv('target_vals_' + case + '_min_invest' + '.csv')
    else:
         results_df.to_csv('target_vals_' + case + '_no_min_invest' + '.csv')       

### Case 2c: Variations in demand (changing starts) and constant generation
Substitute flat demand by introducing two demand peaks and drops each per day with a changing amplitude. In addition to that alter the start of the load shifting processes.

In [ ]:
case = 'dem_variation_starts'

# Data preperation: manipulate demand data
df_data = df_base.copy()
demand = [100] * timesteps

# demand changes
demand[3:4] = [125]
demand[4:5] = [75]

demand[15:16] = [150]
demand[17:18] = [50]

if timesteps > 24:
    demand[26:27] = [25]
    demand[29:30] = [175]

    demand[37:38] = [0]
    demand[41:42] = [200]

df_data['demand_el'] = demand
df_data['Cap_up'] = [100] * timesteps + df_data['Cap_up'] - df_data['demand_el']
df_data['Cap_do'] = [100] * timesteps + df_data['demand_el'] - df_data['Cap_do']

In [ ]:
plot_case(data=df_data, case=case)
plot_case_residual(data=df_data, case=case)

In [ ]:
# Introduce a dict to store the results of every approach
approach_dict = {}

minimum = 199 - existing

for approach in approaches:
    approach_dict[approach] = start_model(df_data, timesteps=len(df_data), plot=True, save=save_figs, case=case, 
                                          shift_interval=24, delay_time=4, efficiency=1, approach=approach,
                                          cost_dsm_up=cost_dsm_up, cost_dsm_down_shift=cost_dsm_down_shift, cost_dsm_down_shed=cost_dsm_down_shed,
                                          addition=False, recovery_time_shift=None, recovery_time_shed=4, figsize=(15,8), 
                                          shed_eligibility=True, shed_time=4, n_yearLimit_shed=6,
                                          introduce_second_dsm_unit=introduce_second_dsm_unit,
                                          fixes=True,
                                          invest=invest,
                                          flex_share_down=flex_share_down,
                                          flex_share_up=flex_share_up,
                                          existing=existing,
                                          ep_costs=0.001, #0
                                          minimum=minimum,
                                          maximum=200,
                                          max_demand_inv=df_data['demand_el'].max(),
                                          max_capacity_down_inv=df_data['demand_el'].max(),
                                          max_capacity_up_inv=200-df_data['demand_el'].min()
                                         )

In [ ]:
show_results_table(approach_dict, save_results)

**Results validation / debugging**

In [ ]:
results_dict = {}
for k, v in approach_dict.items():
    print(f"approach: {k}\t objective: {v[2]['objective']}")
    results_dict[k] = v[2]['objective']
    
results_df = pd.DataFrame.from_dict(results_dict, orient='index')

In [ ]:
if invest:
    for k, v in approach_dict.items():
        print(f"investment in DR for approach {k}: {v[3]} MW of installed capacity")

In [ ]:
if save_results:
    if minimum != 0:
        results_df.to_csv('target_vals_' + case + '_min_invest' + '.csv')
    else:
         results_df.to_csv('target_vals_' + case + '_no_min_invest' + '.csv')       

### Case 2d: Variations in demand (longer duration) and constant generation
Substitute flat demand by introducing in total three demand peaks and drops. The peaks and drops have a longer duration of more than one hour each. Thus, it can be seen how potential limits in interference times (i.e. shift times) behave (for the DLR approach).

> _Note: For the DLR approach, it has been noticed that the core formulation can lead to unbalanced shifts at the end.<br> &rarr; Thus, an own fix is introduced forbidding shifts at the end that cannot be compensated for anymore. This is done within the variant with the addition "\_fixes"._

In [ ]:
case = 'dem_variation_duration'

# Data preperation: manipulate demand data
df_data = df_base.copy()
demand = [100] * timesteps

# demand changes
demand[6:8] = [125] * 2
demand[8:10] = [75] * 2

demand[15:18] = [150] * 3
demand[18:21] = [50] * 3

if timesteps > 24:
    demand[34:38] = [175] * 4
    demand[38:42] = [25] * 4

    #demand[34:36] = [50] * 2
    #demand[36:38] = [0] * 2
    #demand[38:40] = [150] * 2
    #demand[40:42] = [200] * 2

df_data['demand_el'] = demand
df_data['Cap_up'] = [100] * timesteps + df_data['Cap_up'] - df_data['demand_el']
df_data['Cap_do'] = [100] * timesteps + df_data['demand_el'] - df_data['Cap_do']

In [ ]:
plot_case(data=df_data, case=case)
plot_case_residual(data=df_data, case=case)

In [ ]:
# Introduce a dict to store the results of every approach
approach_dict = {}

minimum = 174 - existing

for approach in approaches:
    approach_dict[approach] = start_model(df_data, timesteps=len(df_data), plot=True, 
                                          save=save_figs, case=case, 
                                          shift_interval=24, delay_time=4, efficiency=1, approach=approach,
                                          cost_dsm_up=cost_dsm_up, cost_dsm_down_shift=cost_dsm_down_shift, 
                                          cost_dsm_down_shed=cost_dsm_down_shed,
                                          addition=False, recovery_time_shift=None, recovery_time_shed=4, 
                                          figsize=(15,8),
                                          shed_eligibility=True, shed_time=4, n_yearLimit_shed=6,
                                          introduce_second_dsm_unit=introduce_second_dsm_unit,
                                          invest=invest,
                                          flex_share_down=flex_share_down,
                                          flex_share_up=flex_share_up,
                                          existing=existing,
                                          ep_costs=0.001,
                                          minimum=minimum,
                                          maximum=200,
                                          max_demand_inv=df_data['demand_el'].max(),
                                          max_capacity_down_inv=df_data['demand_el'].max(),
                                          max_capacity_up_inv=200-df_data['demand_el'].min(),
                                         )
    
    if approach == "DLR":
        approach_dict[approach+"_fixes"] = start_model(df_data, timesteps=len(df_data), plot=True, 
                                                      save=save_figs, case=case, 
                                                      shift_interval=24, delay_time=4, efficiency=1, approach=approach,
                                                      cost_dsm_up=cost_dsm_up, cost_dsm_down_shift=cost_dsm_down_shift, 
                                                      cost_dsm_down_shed=cost_dsm_down_shed,
                                                      addition=False, fixes=True,
                                                      recovery_time_shift=None, recovery_time_shed=4, 
                                                      figsize=(15,8),
                                                      shed_eligibility=True, shed_time=4, n_yearLimit_shed=6,
                                                      introduce_second_dsm_unit=introduce_second_dsm_unit,
                                                      invest=invest,
                                                      flex_share_down=flex_share_down,
                                                      flex_share_up=flex_share_up,
                                                      existing=existing,
                                                      ep_costs=0.001,
                                                      minimum=minimum,
                                                      maximum=200,
                                                      max_demand_inv=df_data['demand_el'].max(),
                                                      max_capacity_down_inv=df_data['demand_el'].max(),
                                                      max_capacity_up_inv=200-df_data['demand_el'].min()
                                                      )

In [ ]:
show_results_table(approach_dict, save_results)

**Results validation / debugging**

In [ ]:
results_dict = {}
for k, v in approach_dict.items():
    print(f"approach: {k}\t objective: {v[2]['objective']}")
    results_dict[k] = v[2]['objective']
    
results_df = pd.DataFrame.from_dict(results_dict, orient='index')

In [ ]:
if invest:
    for k, v in approach_dict.items():
        print(f"investment in DR for approach {k}: {v[3]} MW of installed capacity")

In [ ]:
if save_results:
    if minimum != 0:
        results_df.to_csv('target_vals_' + case + '_min_invest' + '.csv')
    else:
         results_df.to_csv('target_vals_' + case + '_no_min_invest' + '.csv')       

### Case 2e: Variations in demand (longer shifts) and constant generation
Substitute flat demand by introducing in total three demand peaks and drops. The time between the initial process and its balancing opposite is much longer in this example.

In [ ]:
case = 'dem_variation_long_shifts'

# Data preperation: manipulate demand data
df_data = df_base.copy()
demand = [100] * timesteps

# demand changes
demand[4:5] = [125]
demand[9:10] = [75]

demand[19:20] = [150]
demand[25:26] = [50]

if timesteps > 24:
    demand[34:35] = [25]
    demand[41:42] = [175]

df_data['demand_el'] = demand
df_data['Cap_up'] = [100] * timesteps + df_data['Cap_up'] - df_data['demand_el']
df_data['Cap_do'] = [100] * timesteps + df_data['demand_el'] - df_data['Cap_do']

In [ ]:
plot_case(data=df_data, case=case)
plot_case_residual(data=df_data, case=case)

In [ ]:
# Introduce a dict to store the results of every approach
approach_dict = {}

minimum = 174 - existing

for approach in approaches:
    approach_dict[approach] = start_model(df_data, timesteps=len(df_data), plot=True, save=save_figs, case=case, 
                                          shift_interval=48, delay_time=4, efficiency=1, approach=approach,
                                          cost_dsm_up=cost_dsm_up, cost_dsm_down_shift=cost_dsm_down_shift, cost_dsm_down_shed=cost_dsm_down_shed,
                                          addition=False, recovery_time_shift=None, recovery_time_shed=4, figsize=(15,8), 
                                          shed_eligibility=True, shed_time=4, n_yearLimit_shed=6,
                                          fixes=True,
                                          introduce_second_dsm_unit=introduce_second_dsm_unit,
                                          invest=invest,
                                          flex_share_down=flex_share_down,
                                          flex_share_up=flex_share_up,
                                          existing=existing,
                                          ep_costs=0.001,
                                          minimum=minimum,
                                          maximum=200,
                                          max_demand_inv=df_data['demand_el'].max(),
                                          max_capacity_down_inv=df_data['demand_el'].max(),
                                          max_capacity_up_inv=200-df_data['demand_el'].min()
                                         )

In [ ]:
show_results_table(approach_dict, save_results)

**Results validation / debugging**

In [ ]:
results_dict = {}
for k, v in approach_dict.items():
    print(f"approach: {k}\t objective: {v[2]['objective']}")
    results_dict[k] = v[2]['objective']
    
results_df = pd.DataFrame.from_dict(results_dict, orient='index')

In [ ]:
if invest:
    for k, v in approach_dict.items():
        print(f"investment in DR for approach {k}: {v[3]} MW of installed capacity")

In [ ]:
if save_results:
    if minimum != 0:
        results_df.to_csv('target_vals_' + case + '_min_invest' + '.csv')
    else:
         results_df.to_csv('target_vals_' + case + '_no_min_invest' + '.csv')       

## Case 3: Increase in demand and constant generation (use case for load shedding)
Substitute flat demand by introducing one demand peak over several hours which gives a use case for load shedding. Load shedding costs which are usually very high are set to a lower value here, in order to force some shedding avtivity.

In [ ]:
case = 'dem_variation_shed'

# Data preperation: manipulate demand data
df_data = df_base.copy()
demand = [100] * timesteps

# demand changes
demand[10:14] = [200] * 4
#demand[30:32] = [0] * 2

df_data['demand_el'] = demand
df_data['Cap_up'] = [100] * timesteps + df_data['Cap_up'] - df_data['demand_el']
df_data['Cap_do'] = [100] * timesteps + df_data['demand_el'] - df_data['Cap_do']

In [ ]:
plot_case(data=df_data, case=case)
plot_case_residual(data=df_data, case=case)

> _**NOTE:** A much lower value is used here for shedding costs in order to incentivize load shedding._

### Case 3a: shift_eligibility = shed_eligibility = True

In [ ]:
# Introduce a dict to store the results of every approach
approach_dict = {}

minimum = 0

for approach in approaches:
    approach_dict[approach] = start_model(df_data, timesteps=len(df_data), plot=True, save=save_figs, case=case, 
                                          shift_interval=24, delay_time=4, efficiency=1, approach=approach,
                                          cost_dsm_up=cost_dsm_up, cost_dsm_down_shift=cost_dsm_down_shift, 
                                          cost_dsm_down_shed=0.1,
                                          addition=False, recovery_time_shift=None, recovery_time_shed=4, 
                                          figsize=(15,8), 
                                          fixes=True,
                                          shift_eligibility=True,
                                          shed_eligibility=True, shed_time=4, n_yearLimit_shed=6,
                                          introduce_second_dsm_unit=introduce_second_dsm_unit,
                                          invest=invest,
                                          flex_share_down=flex_share_down,
                                          flex_share_up=flex_share_up,
                                          existing=existing,
                                          ep_costs=0.001,
                                          minimum=minimum,
                                          maximum=200,
                                          max_demand_inv=df_data['demand_el'].max(),
                                          max_capacity_down_inv=df_data['demand_el'].max(),
                                          max_capacity_up_inv=200-df_data['demand_el'].min()
                                         )

In [ ]:
show_results_table(approach_dict, save_results)

**Results validation / debugging**

In [ ]:
results_dict = {}
for k, v in approach_dict.items():
    print(f"approach: {k}\t objective: {v[2]['objective']}")
    results_dict[k] = v[2]['objective']
    
results_df = pd.DataFrame.from_dict(results_dict, orient='index')

In [ ]:
if invest:
    for k, v in approach_dict.items():
        print(f"investment in DR for approach {k}: {v[3]} MW of installed capacity")

In [ ]:
if save_results:
    if minimum != 0:
        results_df.to_csv('target_vals_' + case + '_min_invest' + '.csv')
    else:
         results_df.to_csv('target_vals_' + case + '_no_min_invest' + '.csv')       

### Case 3b: shift_eligibility = True; shed_eligibility = False

In [ ]:
# Introduce a dict to store the results of every approach
approach_dict = {}

minimum = 0

for approach in approaches:
    approach_dict[approach] = start_model(df_data, timesteps=len(df_data), plot=True, save=save_figs, case=case, 
                                          shift_interval=24, delay_time=4, efficiency=1, approach=approach,
                                          cost_dsm_up=cost_dsm_up, cost_dsm_down_shift=cost_dsm_down_shift, 
                                          cost_dsm_down_shed=0.1,
                                          addition=False, recovery_time_shift=None, recovery_time_shed=4, 
                                          figsize=(15,8), 
                                          fixes=True,
                                          shift_eligibility=True,
                                          shed_eligibility=False, shed_time=4, n_yearLimit_shed=6,
                                          introduce_second_dsm_unit=introduce_second_dsm_unit,
                                          invest=invest,
                                          flex_share_down=flex_share_down,
                                          flex_share_up=flex_share_up,
                                          existing=existing,
                                          ep_costs=0.001,
                                          minimum=minimum,
                                          maximum=200,
                                          max_demand_inv=df_data['demand_el'].max(),
                                          max_capacity_down_inv=df_data['demand_el'].max(),
                                          max_capacity_up_inv=200-df_data['demand_el'].min()
                                         )

In [ ]:
show_results_table(approach_dict, save_results)

**Results validation / debugging**

In [ ]:
results_dict = {}
for k, v in approach_dict.items():
    print(f"approach: {k}\t objective: {v[2]['objective']}")
    results_dict[k] = v[2]['objective']
    
results_df = pd.DataFrame.from_dict(results_dict, orient='index')

In [ ]:
if invest:
    for k, v in approach_dict.items():
        print(f"investment in DR for approach {k}: {v[3]} MW of installed capacity")

In [ ]:
if save_results:
    if minimum != 0:
        results_df.to_csv('target_vals_' + case + '_min_invest' + '.csv')
    else:
         results_df.to_csv('target_vals_' + case + '_no_min_invest' + '.csv')       

### Case 3c: shift_eligibility = False; shed_eligibility = True

In [ ]:
# Introduce a dict to store the results of every approach
approach_dict = {}

minimum = 0

for approach in approaches:
    approach_dict[approach] = start_model(df_data, timesteps=len(df_data), plot=True, save=save_figs, case=case, 
                                          shift_interval=24, delay_time=4, efficiency=1, approach=approach,
                                          cost_dsm_up=cost_dsm_up, cost_dsm_down_shift=cost_dsm_down_shift, 
                                          cost_dsm_down_shed=0.1,
                                          addition=False, recovery_time_shift=None, recovery_time_shed=4, 
                                          figsize=(15,8), 
                                          fixes=True,
                                          shift_eligibility=False,
                                          shed_eligibility=True, shed_time=4, n_yearLimit_shed=6,
                                          introduce_second_dsm_unit=introduce_second_dsm_unit,
                                          invest=invest,
                                          flex_share_down=flex_share_down,
                                          flex_share_up=flex_share_up,
                                          existing=existing,
                                          ep_costs=0.001,
                                          minimum=minimum,
                                          maximum=200,
                                          max_demand_inv=df_data['demand_el'].max(),
                                          max_capacity_down_inv=df_data['demand_el'].max(),
                                          max_capacity_up_inv=200-df_data['demand_el'].min()
                                         )

In [ ]:
show_results_table(approach_dict, save_results)

**Results validation / debugging**

In [ ]:
results_dict = {}
for k, v in approach_dict.items():
    print(f"approach: {k}\t objective: {v[2]['objective']}")
    results_dict[k] = v[2]['objective']
    
results_df = pd.DataFrame.from_dict(results_dict, orient='index')

In [ ]:
if invest:
    for k, v in approach_dict.items():
        print(f"investment in DR for approach {k}: {v[3]} MW of installed capacity")

In [ ]:
if save_results:
    if minimum != 0:
        results_df.to_csv('target_vals_' + case + '_min_invest' + '.csv')
    else:
         results_df.to_csv('target_vals_' + case + '_no_min_invest' + '.csv')       

## Case 4: Variations in generation and constant demand
Introduce wind peaks and wind drops with a changing amplitude.

In [ ]:
case = 'gen_variation'

# Data preperation: manipulate wind data
df_data = df_base.copy()
wind = [100] * timesteps

# wind changes
wind[1:2] = [0]
wind[4:5] = [175]
if timesteps > 24:
    wind[41:42] = [200]
    wind[44:45] = [25]

df_data['wind'] = wind

In [ ]:
plot_case(data=df_data, case=case)
plot_case_residual(data=df_data, case=case)

In [ ]:
# Introduce a dict to store the results of every approach
approach_dict = {}

if force_min_invest:
    minimum = 0

for approach in approaches:
    approach_dict[approach] = start_model(df_data, timesteps=len(df_data), plot=True, save=save_figs, case=case, 
                                          shift_interval=24, delay_time=4, efficiency=1, approach=approach,
                                          cost_dsm_up=cost_dsm_up, cost_dsm_down_shift=cost_dsm_down_shift, cost_dsm_down_shed=cost_dsm_down_shed,
                                          addition=False, recovery_time_shift=None, recovery_time_shed=4, figsize=(15,8), 
                                          shed_eligibility=True, shed_time=4, n_yearLimit_shed=6,
                                          fixes=True,
                                          introduce_second_dsm_unit=introduce_second_dsm_unit,
                                          invest=invest,
                                          flex_share_down=flex_share_down,
                                          flex_share_up=flex_share_up,
                                          existing=existing,
                                          ep_costs=0.001,
                                          minimum=minimum,
                                          maximum=200,
                                          max_demand_inv=df_data['demand_el'].max(),
                                          max_capacity_down_inv=df_data['demand_el'].max(),
                                          max_capacity_up_inv=200-df_data['demand_el'].min()
                                         )

In [ ]:
show_results_table(approach_dict, save_results)

**Results validation / debugging**

In [ ]:
results_dict = {}
for k, v in approach_dict.items():
    print(f"approach: {k}\t objective: {v[2]['objective']}")
    results_dict[k] = v[2]['objective']
    
results_df = pd.DataFrame.from_dict(results_dict, orient='index')

In [ ]:
if invest:
    for k, v in approach_dict.items():
        print(f"investment in DR for approach {k}: {v[3]} MW of installed capacity")

In [ ]:
if save_results:
    if minimum != 0:
        results_df.to_csv('target_vals_' + case + '_min_invest' + '.csv')
    else:
         results_df.to_csv('target_vals_' + case + '_no_min_invest' + '.csv')       

## Case 5: Variations in demand and generation
Demand and wind variations are introduced. The residual load plot shows what is effectively to be balanced here.

In [ ]:
case = 'combined_variation'

# Data preperation: manipulate demand and wind data
df_data = df_base.copy()
demand = [100] * timesteps
wind = [100] * timesteps

# demand changes
demand[11:12] = [50]
demand[13:14] = [150]
if timesteps > 24:
    demand[33:34] = [150]
    demand[37:38] = [50]

# wind changes
wind[3:4] = [0]
wind[10:11] = [175]
if timesteps > 24:
    wind[38:39] = [200]
    wind[40:41] = [25]

df_data['demand_el'] = demand
df_data['Cap_up'] = [100] * timesteps + df_data['Cap_up'] - df_data['demand_el']
df_data['Cap_do'] = [100] * timesteps + df_data['demand_el'] - df_data['Cap_do']
df_data['wind'] = wind

In [ ]:
plot_case(data=df_data, case=case)
plot_case_residual(data=df_data, case=case)

In [ ]:
# Introduce a dict to store the results of every approach
approach_dict = {}

if force_min_invest:
    minimum = 199 - existing

for approach in approaches:
    approach_dict[approach] = start_model(df_data, timesteps=len(df_data), plot=True, save=save_figs, case=case, 
                                          shift_interval=24, delay_time=4, efficiency=1, approach=approach,
                                          cost_dsm_up=cost_dsm_up, cost_dsm_down_shift=cost_dsm_down_shift, cost_dsm_down_shed=cost_dsm_down_shed,
                                          addition=False, recovery_time_shift=None, recovery_time_shed=4, figsize=(15,8), 
                                          shed_eligibility=True, shed_time=4, n_yearLimit_shed=6,
                                          fixes=True,
                                          introduce_second_dsm_unit=introduce_second_dsm_unit,
                                          invest=invest,
                                          flex_share_down=flex_share_down,
                                          flex_share_up=flex_share_up,
                                          existing=existing,
                                          ep_costs=0.001,
                                          minimum=minimum,
                                          maximum=200,
                                          max_demand_inv=df_data['demand_el'].max(),
                                          max_capacity_down_inv=df_data['demand_el'].max(),
                                          max_capacity_up_inv=200-df_data['demand_el'].min()
                                         )

In [ ]:
show_results_table(approach_dict, save_results)

**Results validation / debugging**

In [ ]:
results_dict = {}
for k, v in approach_dict.items():
    print(f"approach: {k}\t objective: {v[2]['objective']}")
    results_dict[k] = v[2]['objective']
    
results_df = pd.DataFrame.from_dict(results_dict, orient='index')

In [ ]:
if invest:
    for k, v in approach_dict.items():
        print(f"investment in DR for approach {k}: {v[3]} MW of installed capacity")

In [ ]:
if save_results:
    if minimum != 0:
        results_df.to_csv('target_vals_' + case + '_min_invest' + '.csv')
    else:
         results_df.to_csv('target_vals_' + case + '_no_min_invest' + '.csv')       

# Parameters and constraints variations
* Parameters to be variated:
    * delay times
    * shift times
    * costs
    * efficiency
    * longer term energy limits: recovery time and maximum activations
    * short-term energy limits: recovery time and short term limit
* Constraints to be analyzed:
    * optional constraints for Gils (2015) &rarr; included in long- and short-term energy limits analysis
    * overall energy limits (activate / deactive) &rarr; see in long-term energy limits analysis
    * additional constraint (eq. 10 from Zerrahn & Schill 2015)
 
The cases for the parameter and constraints variations are chosen in such a way that the effects of parameter and constraints variations become visible. Therefore, the model setting and constraints formulation has been carefully analyzed in order to find out, how the model setting and individual parameters and constraints should behave.

## Variations in delay time

### Define settings for delay time analysis

In [ ]:
case = 'dem_variation'

# Data preperation: manipulate demand data
df_data = df_base.copy()
demand = [100] * timesteps

# demand changes
demand[4:5] = [125]
demand[5:6] = [75]

demand[11:12] = [150]
demand[13:14] = [50]

if timesteps > 24:
    demand[26:27] = [25]
    demand[29:30] = [175]

    demand[36:37] = [0]
    demand[40:41] = [200]

df_data['demand_el'] = demand
df_data['Cap_up'] = [100] * timesteps + df_data['Cap_up'] - df_data['demand_el']
df_data['Cap_do'] = [100] * timesteps + df_data['demand_el'] - df_data['Cap_do']

In [ ]:
plot_case(data=df_data, case='dem_variation')
plot_case_residual(data=df_data, case='dem_variation')

### Compare variations in delay time

In [ ]:
# Introduce a dict to store the results of every approach
delay_times = [1, 2, 3, 4, 5]
approach_dict = {}

for dt in delay_times:
    print("-----------------------------------------------------------------------------------------------")
    print("delay time {:d}".format(dt))
    print("-----------------------------------------------------------------------------------------------")
    for approach in approaches:

        approach_dict[(approach, dt)] = \
            start_model(df_data, 
                        timesteps=len(df_data), 
                        plot=True, 
                        save=save_figs, 
                        case=case,
                        shift_interval=24, 
                        param_variations='_delay_time_{:d}'.format(dt), 
                        delay_time=dt, 
                        efficiency=1, 
                        approach=approach,
                        cost_dsm_up=cost_dsm_up, 
                        cost_dsm_down_shift=cost_dsm_down_shift, 
                        cost_dsm_down_shed=cost_dsm_down_shed,
                        addition=False, 
                        recovery_time_shift=None, 
                        recovery_time_shed=4, 
                        figsize=(15,8), 
                        shed_eligibility=True, 
                        shed_time=4, 
                        n_yearLimit_shed=6,
                        fixes=True,
                        introduce_second_dsm_unit=introduce_second_dsm_unit,
                        invest=invest,
                        flex_share_down=flex_share_down,
                        flex_share_up=flex_share_up,
                        ep_costs=0.001,
                        minimum=minimum,
                        maximum=200,
                        max_demand_inv=df_data['demand_el'].max(),
                        max_capacity_down_inv=df_data['demand_el'].max(),
                        max_capacity_up_inv=200-df_data['demand_el'].min())

In [ ]:
show_results_table(approach_dict, save_results, MultiIndex=True,
                  param_name="delay_time")

In [ ]:
results_dict = {}
for k, v in approach_dict.items():
    print(f"approach: {k}\t objective: {v[2]['objective']}")
    results_dict[k] = v[2]['objective']
    
results_df = pd.DataFrame.from_dict(results_dict, orient='index')

In [ ]:
if invest:
    for k, v in approach_dict.items():
        print(f"investment in DR for approach {k}: {v[3]} MW of installed capacity")

## Variations in shift time

### Define settings for shift time analysis

In [ ]:
case = 'dem_variation_duration'

# Data preperation: manipulate demand data
df_data = df_base.copy()
demand = [100] * timesteps

# demand changes
demand[6:8] = [125] * 2
demand[8:10] = [75] * 2

demand[15:18] = [150] * 3
demand[18:21] = [50] * 3

if timesteps > 24:
    demand[34:38] = [0] * 4
    demand[38:42] = [200] * 4

df_data['demand_el'] = demand
df_data['Cap_up'] = [100] * timesteps + df_data['Cap_up'] - df_data['demand_el']
df_data['Cap_do'] = [100] * timesteps + df_data['demand_el'] - df_data['Cap_do']

In [ ]:
plot_case(data=df_data, case='dem_variation_duration')
plot_case_residual(data=df_data, case='dem_variation_duration')

### Compare variations in shift time

In [ ]:
# Introduce a dict to store the results of every approach
shift_times = [1, 2, 3, 4, 5]
approach_dict = {}

for st in shift_times:
    print("-----------------------------------------------------------------------------------------------")
    print("shift time {:d}".format(st))
    print("-----------------------------------------------------------------------------------------------")
    for approach in approaches:

        approach_dict[(approach, st)] = \
            start_model(df_data, timesteps=len(df_data), plot=True, save=save_figs, case=case,
                        shift_interval=24, 
                        param_variations='_shift_time_{:d}'.format(st),
                        method='delay', delay_time=4, efficiency=1, approach=approach,
                        cost_dsm_up=cost_dsm_up, cost_dsm_down_shift=cost_dsm_down_shift, 
                        cost_dsm_down_shed=cost_dsm_down_shed,
                        addition=False, recovery_time_shift=None, recovery_time_shed=4, figsize=(15,8), 
                        shed_eligibility=True, shed_time=4, n_yearLimit_shed=6,
                        introduce_second_dsm_unit=introduce_second_dsm_unit,
                        shift_time=st,
                        invest=invest,
                        flex_share_down=flex_share_down,
                        flex_share_up=flex_share_up,
                        ep_costs=0.001,
                        minimum=minimum,
                        maximum=200,
                        max_demand_inv=df_data['demand_el'].max(),
                        max_capacity_down_inv=df_data['demand_el'].max(),
                        max_capacity_up_inv=200-df_data['demand_el'].min())
        
        if approach == "DLR":
            approach_dict[(approach, st)] = \
                start_model(df_data, timesteps=len(df_data), plot=True, save=save_figs, case=case,
                            shift_interval=24, 
                            param_variations='_fixes_shift_time_{:d}'.format(st),
                            method='delay', delay_time=4, efficiency=1, approach=approach,
                            cost_dsm_up=cost_dsm_up, cost_dsm_down_shift=cost_dsm_down_shift, 
                            cost_dsm_down_shed=cost_dsm_down_shed,
                            addition=False, fixes=True,
                            recovery_time_shift=None, recovery_time_shed=4, figsize=(15,8), 
                            shed_eligibility=True, shed_time=4, n_yearLimit_shed=6,
                            introduce_second_dsm_unit=introduce_second_dsm_unit,
                            shift_time=st,
                            invest=invest,
                            flex_share_down=flex_share_down,
                            flex_share_up=flex_share_up,
                            ep_costs=0.001,
                            minimum=minimum,
                            maximum=200,
                            max_demand_inv=df_data['demand_el'].max(),
                            max_capacity_down_inv=df_data['demand_el'].max(),
                            max_capacity_up_inv=200-df_data['demand_el'].min())

In [ ]:
show_results_table(approach_dict, save_results, MultiIndex=True,
                  param_name="shift_time")

In [ ]:
results_dict = {}
for k, v in approach_dict.items():
    print(f"approach: {k}\t objective: {v[2]['objective']}")
    results_dict[k] = v[2]['objective']
    
results_df = pd.DataFrame.from_dict(results_dict, orient='index')

In [ ]:
if invest:
    for k, v in approach_dict.items():
        print(f"investment in DR for approach {k}: {v[3]} MW of installed capacity")

## Variations in DSM costs

### Define settings for cost analysis

In [ ]:
case = 'dem_variation'

# Data preperation: manipulate demand data
df_data = df_base.copy()
demand = [100] * timesteps

# demand changes
demand[4:5] = [125]
demand[5:6] = [75]

demand[11:12] = [150]
demand[13:14] = [50]

if timesteps > 24:
    demand[26:27] = [25]
    demand[29:30] = [175]

    demand[36:37] = [0]
    demand[40:41] = [200]

df_data['demand_el'] = demand
df_data['Cap_up'] = [100] * timesteps + df_data['Cap_up'] - df_data['demand_el']
df_data['Cap_do'] = [100] * timesteps + df_data['demand_el'] - df_data['Cap_do']

In [ ]:
plot_case(data=df_data, case='dem_variation')
plot_case_residual(data=df_data, case='dem_variation')

### Compare variations in costs

In [ ]:
# Introduce a dict to store the results of every approach
cost_dsm = [0.1, 1, 16.74*2, 16.75*2, 32.5*2, 200]
approach_dict = {}

for cost in cost_dsm:
    
    # Cost is split half on upwards and downwards shift; shedding gets high costs
    cost_dsm_up = cost/2
    cost_dsm_down_shift = cost/2
    cost_dsm_down_shed = 1000 * cost

    print("-----------------------------------------------------------------------------------------------")
    print("costs {:.4f}".format(cost))
    print("-----------------------------------------------------------------------------------------------")
    for approach in approaches:

        approach_dict[(approach, cost)] = \
            start_model(df_data, timesteps=len(df_data), plot=True, save=save_figs, case=case,
                        shift_interval=24, 
                        param_variations='_dsm_cost_{:f}'.format(cost_dsm_up),
                        method='delay', delay_time=4, efficiency=1, approach=approach,
                        cost_dsm_up=cost_dsm_up, cost_dsm_down_shift=cost_dsm_down_shift, cost_dsm_down_shed=cost_dsm_down_shed,
                        addition=False, recovery_time_shift=None, recovery_time_shed=4, figsize=(15,8), 
                        shed_eligibility=True, shed_time=4, n_yearLimit_shed=6,
                        introduce_second_dsm_unit=introduce_second_dsm_unit,
                        invest=invest,
                        flex_share_down=flex_share_down,
                        flex_share_up=flex_share_up,
                        ep_costs=0.001,
                        minimum=minimum,
                        maximum=200,
                        max_demand_inv=df_data['demand_el'].max(),
                        max_capacity_down_inv=df_data['demand_el'].max(),
                        max_capacity_up_inv=200-df_data['demand_el'].min())

In [ ]:
show_results_table(approach_dict, save_results, MultiIndex=True,
                  param_name="costs")

In [ ]:
results_dict = {}
for k, v in approach_dict.items():
    print(f"approach: {k}\t objective: {v[2]['objective']}")
    results_dict[k] = v[2]['objective']
    
results_df = pd.DataFrame.from_dict(results_dict, orient='index')

In [ ]:
if invest:
    for k, v in approach_dict.items():
        print(f"investment in DR for approach {k}: {v[3]} MW of installed capacity")

**Reset costs values:** Reset DSM costs to original values for further considerations

In [ ]:
cost_dsm = 0.1

# Cost is split half on upwards and downwards shift; shedding gets high costs
cost_dsm_up = cost_dsm/2
cost_dsm_down_shift = cost_dsm/2
cost_dsm_down_shed = 1000 * cost_dsm

## Variations in DSM efficiency
So far, only examples with an efficiency of one, i.e. no losses have been considered.<br>
If efficiencies are introduces, load shifts can be balanced out except for these losses.

> _Note:_
> * _For the DLR approach two different variants are considered: the basic one "core" and another one including own fixes. &rarr; Here, the added condition forbidding unbalanced load shifts prevents unexpected behaviour._

### Define settings for efficiency analysis

In [ ]:
case = 'dem_variation_eff'

# Data preperation: manipulate demand data
df_data = df_base.copy()
demand = [100] * timesteps

# demand changes
demand[4:5] = [125]
demand[5:6] = [75]

demand[11:12] = [150]
demand[13:14] = [50]
    
if timesteps > 24:
    demand[26:27] = [25]
    demand[29:30] = [175]

    demand[36:37] = [0]
    demand[40:41] = [200]

    demand[43:44] = [0]
    demand[45:46] = [200]

df_data['demand_el'] = demand
df_data['Cap_up'] = [100] * timesteps + df_data['Cap_up'] - df_data['demand_el']
df_data['Cap_do'] = [100] * timesteps + df_data['demand_el'] - df_data['Cap_do']

In [ ]:
plot_case(data=df_data, case='dem_variation')
plot_case_residual(data=df_data, case='dem_variation')

### Compare variations in efficiency

In [ ]:
# Introduce a dict to store the results of every approach
efficiencies = [1, 0.95, 0.8, 0.2]
approach_dict = {}

for eff in efficiencies:
    
    print("-----------------------------------------------------------------------------------------------")
    print("efficiency {:.2f}".format(eff))
    print("-----------------------------------------------------------------------------------------------")
    
    eff_assigned = eff
    for approach in approaches:
        
        approach_dict[(approach, eff)] = \
            start_model(df_data, timesteps=len(df_data), plot=True, save=save_figs, case=case,
                        shift_interval=24, 
                        param_variations='_efficiency_{:f}'.format(eff),
                        method='delay', delay_time=4, efficiency=eff_assigned, approach=approach,
                        cost_dsm_up=cost_dsm_up, cost_dsm_down_shift=cost_dsm_down_shift, 
                        cost_dsm_down_shed=cost_dsm_down_shed,
                        addition=False, recovery_time_shift=None, recovery_time_shed=4, figsize=(15,8), 
                        shed_eligibility=True, shed_time=4, n_yearLimit_shed=6,
                        introduce_second_dsm_unit=introduce_second_dsm_unit,
                        invest=invest,
                        flex_share_down=flex_share_down,
                        flex_share_up=flex_share_up,
                        ep_costs=0.001,
                        minimum=minimum,
                        maximum=200,
                        max_demand_inv=df_data['demand_el'].max(),
                        max_capacity_down_inv=df_data['demand_el'].max(),
                        max_capacity_up_inv=200-df_data['demand_el'].min())
        
        # variant with own fixes
        if approach == "DLR":            
            approach_dict[(approach+"_fixes", eff)] = \
                start_model(df_data, timesteps=len(df_data), plot=True, save=save_figs, case=case,
                            shift_interval=24, 
                            param_variations='_fixes_efficiency_{:f}'.format(eff),
                            method='delay', delay_time=4, efficiency=eff_assigned, approach=approach,
                            cost_dsm_up=cost_dsm_up, cost_dsm_down_shift=cost_dsm_down_shift, 
                            cost_dsm_down_shed=cost_dsm_down_shed,
                            addition=False, fixes=True,
                            recovery_time_shift=None, recovery_time_shed=4, figsize=(15,8), 
                            shed_eligibility=True, shed_time=4, n_yearLimit_shed=6,
                            introduce_second_dsm_unit=introduce_second_dsm_unit,
                            invest=invest,
                            flex_share_down=flex_share_down,
                            flex_share_up=flex_share_up,
                            ep_costs=0.001,
                            minimum=minimum,
                            maximum=200,
                            max_demand_inv=df_data['demand_el'].max(),
                            max_capacity_down_inv=df_data['demand_el'].max(),
                            max_capacity_up_inv=200-df_data['demand_el'].min())          

In [ ]:
show_results_table(approach_dict, save_results, MultiIndex=True,
                  param_name="efficiency")

In [ ]:
results_dict = {}
for k, v in approach_dict.items():
    print(f"approach: {k}\t objective: {v[2]['objective']}")
    results_dict[k] = v[2]['objective']
    
results_df = pd.DataFrame.from_dict(results_dict, orient='index')

In [ ]:
if invest:
    for k, v in approach_dict.items():
        print(f"investment in DR for approach {k}: {v[3]} MW of installed capacity")

## Variations in recovery time / max. activations

The approaches differ in the way, a recovery time resp. a limit for a given time frame is set. The following elements are used for depicting this.

| approach | parameter used | description |
| ---- | ---- | ---- |
| DIW | recovery time | defines the time between the end of a shifting process and the begin of a subsequent one |
| DLR | $n_{YearLimit}$ | defines the maximum amount of shifts within the optimization timeframe (one year) |

The following anologies can be identified which are used to harmonize parameterization:

$$ \qquad n_{YearLimit} = \frac{T} {t_{delay} + t_{recovery}}$$

where

$ T $ : overall optimization timeframe (in realistic examples usually one year)<br>

> _Note:_<br>
> _For the DLR approach two different variants are considered: the basic one "core" and another one including own fixes. &rarr; Here, the added conditions preventing shifts which cannot be balanced within the optimization timeframe prevent unexpected behaviour._

### Basic variant
Use the same generation and demand patterns as for most of the other parameter variations above (except for one extra shift at the end).

In [ ]:
# Introduce a dict to store the results of every approach
recovery_time = [None, 2, 4, 16, 44]
approach_dict = {}

for rt in recovery_time:
    
    if rt is not None:
        param_variations='_recovery_time_{:d}'.format(rt)
    else:
        param_variations='_no_recovery_time'

    print("-----------------------------------------------------------------------------------------------")
    if not rt is None:
        print("recovery time {:d}; max. activations: {:.0f}".format(rt, len(df_data)//(4+rt)))
    else:
        print("no limit on recovery time")
    print("-----------------------------------------------------------------------------------------------")
    for approach in approaches:            
        
        approach_dict[(approach, rt)] = \
            start_model(df_data, timesteps=len(df_data), plot=True, save=save_figs, case=case,
                        shift_interval=24, 
                        param_variations=param_variations,
                        method='delay', delay_time=4, efficiency=1, approach=approach,
                        cost_dsm_up=cost_dsm_up, cost_dsm_down_shift=cost_dsm_down_shift, cost_dsm_down_shed=cost_dsm_down_shed,
                        addition=False, recovery_time_shift=rt, recovery_time_shed=4, figsize=(15,8), 
                        shed_eligibility=True, shed_time=4, n_yearLimit_shed=6,
                        ActivateYearLimit=True,
                        introduce_second_dsm_unit=introduce_second_dsm_unit,
                        invest=invest,
                        flex_share_down=flex_share_down,
                        flex_share_up=flex_share_up,
                        ep_costs=0.001,
                        minimum=minimum,
                        maximum=200,
                        max_demand_inv=df_data['demand_el'].max(),
                        max_capacity_down_inv=df_data['demand_el'].max(),
                        max_capacity_up_inv=200-df_data['demand_el'].min())

In [ ]:
show_results_table(approach_dict, save_results, MultiIndex=True)

In [ ]:
results_dict = {}
for k, v in approach_dict.items():
    print(f"approach: {k}\t objective: {v[2]['objective']}")
    results_dict[k] = v[2]['objective']
    
results_df = pd.DataFrame.from_dict(results_dict, orient='index')

In [ ]:
if invest:
    for k, v in approach_dict.items():
        print(f"investment in DR for approach {k}: {v[3]} MW of installed capacity")

### Additional variant
Show dependency of recovery time on consumption pattern for DIW and DLR approach:
* _DIW approach_: Since recovery time constraint is introduced only for upshifts and cap_up, it restricts load shifting further if initial load increases are needed to balance (demand) variations.
* _DLR approach_: Effectively an energy limit is posed for the initial shifts. If all initial shifts are downshifts, the energy limit should become binding.
* This can be identified by adapting case 2b from above in the following way:
    1. The order of the demand variation for the third and fourth use case of load shifting in case 2b is switched, i.e. an initial load decrease is now needed.
    2. Change the amplitudes: Slightly increase it for the first shift cycle and change the amplitudes for second and fourth shifting cycle.

In [ ]:
case = 'dem_variation_switched'

# Data preperation: manipulate demand data
df_data = df_base.copy()
demand = [100] * timesteps

# demand changes
demand[4:5] = [150]
demand[5:6] = [50]

demand[11:12] = [200]
demand[13:14] = [0]

#demand[17:18] = [200]
#demand[20:21] = [0]

if timesteps > 24:
    demand[26:27] = [150]
    demand[29:30] = [50]

    #demand[31:32] = [150]
    #demand[35:36] = [50]    
    
    demand[36:37] = [175]
    demand[40:41] = [25]

df_data['demand_el'] = demand
df_data['Cap_up'] = [100] * timesteps + df_data['Cap_up'] - df_data['demand_el']
df_data['Cap_do'] = [100] * timesteps + df_data['demand_el'] - df_data['Cap_do']

In [ ]:
plot_case(data=df_data, case='dem_variation_switched')
plot_case_residual(data=df_data, case='dem_variation_switched')

In [ ]:
# Introduce a dict to store the results of every approach
recovery_time = [None, 2, 4, 16, 44]
approach_dict = {}

for rt in recovery_time:
    
    if rt is not None:
        param_variations='_recovery_time_{:d}'.format(rt)
        param_variations_fixes='_fixes_recovery_time_{:d}'.format(rt)
    else:
        param_variations='_no_recovery_time'
        param_variations_fixes='_fixes_no_recovery_time'

    print("-----------------------------------------------------------------------------------------------")
    if not rt is None:
        print("recovery time {:d}; max. activations: {:.0f}; cumulative shift time {:.0f}".format(rt, 
            len(df_data)//(4+rt), len(df_data)//(4+rt) * 2))
    else:
        print("no limit on recovery time")
    print("-----------------------------------------------------------------------------------------------")
    for approach in approaches:            
        
        approach_dict[(approach, rt)] = \
            start_model(df_data, timesteps=len(df_data), plot=True, save=save_figs, case=case,
                        shift_interval=24, 
                        param_variations=param_variations,
                        method='delay', delay_time=4, efficiency=1, approach=approach,
                        cost_dsm_up=cost_dsm_up, cost_dsm_down_shift=cost_dsm_down_shift, 
                        cost_dsm_down_shed=cost_dsm_down_shed,
                        addition=False, recovery_time_shift=rt, recovery_time_shed=4, figsize=(15,8), 
                        shed_eligibility=True, shed_time=4, n_yearLimit_shed=6,
                        ActivateYearLimit=True,
                        introduce_second_dsm_unit=introduce_second_dsm_unit,
                        invest=invest,
                        flex_share_down=flex_share_down,
                        flex_share_up=flex_share_up,
                        ep_costs=0.001,
                        minimum=minimum,
                        maximum=200,
                        max_demand_inv=df_data['demand_el'].max(),
                        max_capacity_down_inv=df_data['demand_el'].max(),
                        max_capacity_up_inv=200-df_data['demand_el'].min())
        
        # variant with own fixes
        if approach == "DLR":            
            approach_dict[(approach+"_fixes", rt)] = \
                start_model(df_data, timesteps=len(df_data), plot=True, save=save_figs, case=case,
                            shift_interval=24, 
                            param_variations=param_variations_fixes,
                            method='delay', delay_time=4, efficiency=1, approach=approach,
                            cost_dsm_up=cost_dsm_up, cost_dsm_down_shift=cost_dsm_down_shift, 
                            cost_dsm_down_shed=cost_dsm_down_shed,
                            addition=False, fixes=True,
                            recovery_time_shift=rt, recovery_time_shed=4, figsize=(15,8), 
                            shed_eligibility=True, shed_time=4, n_yearLimit_shed=6,
                            ActivateYearLimit=True,
                            introduce_second_dsm_unit=introduce_second_dsm_unit,
                            invest=invest,
                            flex_share_down=flex_share_down,
                            flex_share_up=flex_share_up,
                            ep_costs=0.001,
                            minimum=minimum,
                            maximum=200,
                            max_demand_inv=df_data['demand_el'].max(),
                            max_capacity_down_inv=df_data['demand_el'].max(),
                            max_capacity_up_inv=200-df_data['demand_el'].min())

In [ ]:
show_results_table(approach_dict, save_results, MultiIndex=True)

In [ ]:
results_dict = {}
for k, v in approach_dict.items():
    print(f"approach: {k}\t objective: {v[2]['objective']}")
    results_dict[k] = v[2]['objective']
    
results_df = pd.DataFrame.from_dict(results_dict, orient='index')

In [ ]:
if invest:
    for k, v in approach_dict.items():
        print(f"investment in DR for approach {k}: {v[3]} MW of installed capacity")

### Compare DIW and DLR approach separately (recovery_time and t_dayLimit)
Dependent on how parameterization of the parameters recovery_time from DIW and t_dayLimit from DLR is set, they can be used to model more or less the same thing, i.e. and energy limit imposed over a certain (smaller) timeframe. Hence, an additional comparison is made in the following where only the two approaches are compared to each other.

In [ ]:
# Introduce a dict to store the results of every approach
recovery_time = [2, 4, 16, 44]
approach_dict = {}

for rt in recovery_time:
    
    if rt is not None:
        param_variations='_recovery_time_{:d}'.format(rt)
    else:
        param_variations='_no_recovery_time'

    print("-----------------------------------------------------------------------------------------------")
    if not rt is None:
        print("recovery time resp. t_dayLimit {:d}".format(rt))
    else:
        print("no limit on recovery time resp. t_dayLimit")
    print("-----------------------------------------------------------------------------------------------")
    for approach in approaches:
        
        if approach in ["DIW", "DLR"]:
        
            approach_dict[(approach, rt)] = \
                start_model(df_data, timesteps=len(df_data), plot=True, save=save_figs, case=case,
                            shift_interval=24, 
                            param_variations=param_variations,
                            method='delay', delay_time=4, efficiency=1, approach=approach,
                            cost_dsm_up=cost_dsm_up, cost_dsm_down_shift=cost_dsm_down_shift, 
                            cost_dsm_down_shed=cost_dsm_down_shed,
                            addition=False, fixes=True,
                            recovery_time_shift=rt, recovery_time_shed=4, figsize=(15,8), 
                            shed_eligibility=True, shed_time=4, n_yearLimit_shed=6,
                            ActivateYearLimit=False, ActivateDayLimit=True,
                            t_dayLimit = rt, 
                            introduce_second_dsm_unit=introduce_second_dsm_unit,
                            invest=invest,
                            flex_share_down=flex_share_down,
                            flex_share_up=flex_share_up,
                            ep_costs=0.001,
                            minimum=minimum,
                            maximum=200,
                            max_demand_inv=df_data['demand_el'].max(),
                            max_capacity_down_inv=df_data['demand_el'].max(),
                            max_capacity_up_inv=200-df_data['demand_el'].min())

In [ ]:
show_results_table(approach_dict, save_results, MultiIndex=True,
                  param_name="recovery_time")

In [ ]:
results_dict = {}
for k, v in approach_dict.items():
    print(f"approach: {k}\t objective: {v[2]['objective']}")
    results_dict[k] = v[2]['objective']
    
results_df = pd.DataFrame.from_dict(results_dict, orient='index')

In [ ]:
if invest:
    for k, v in approach_dict.items():
        print(f"investment in DR for approach {k}: {v[3]} MW of installed capacity")

## Variations in amount of daily activations

### Define settings for daily constraint analysis

In [ ]:
case = 'dem_variation_daily'

# Data preperation: manipulate demand data
df_data = df_base.copy()
demand = [100] * timesteps

# demand changes
demand[4:5] = [175]
demand[5:6] = [25]

demand[11:12] = [200]
demand[13:14] = [0]

demand[17:18] = [200]
demand[19:20] = [0]

if timesteps > 24:
    demand[26:27] = [25]
    demand[29:30] = [175]

    demand[36:37] = [0]
    demand[40:41] = [200]

df_data['demand_el'] = demand
df_data['Cap_up'] = [100] * timesteps + df_data['Cap_up'] - df_data['demand_el']
df_data['Cap_do'] = [100] * timesteps + df_data['demand_el'] - df_data['Cap_do']

In [ ]:
plot_case(data=df_data, case=case)
plot_case_residual(data=df_data, case=case)

### Compare variations in amount of daily activations

For the DLR approach from Gils (2015) a daily limit is optional.

The following elements are used for depicting this.

| approach | parameter used | description |
| ---- | ---- | ---- |
| DLR | $t_{DayLimit}$ | defines the maximum time for up- resp. downshifts, i.e. the maximum amount of subsequent hours for up- resp. downshifts (not including the current hour) |

The following anology can be identified which is used to harmonize parameterization:

$$ \qquad t_{DayLimit} =  \frac{24}{f_d}-1$$

In [ ]:
# Introduce a dict to store the results of every approach
day_limit = [24, 6, 2, 1]
approach_dict = {}

for dl in day_limit:
    
    param_variations='_day_limit_{:d}'.format(dl)

    print("-----------------------------------------------------------------------------------------------")
    print("day limit {:d}; t_DayLimit: {:d}".format(dl, int(24/dl-1)))
    print("-----------------------------------------------------------------------------------------------")
    for approach in approaches:
        if approach == "DLR":
        
            approach_dict[(approach, dl)] = \
                start_model(df_data, timesteps=len(df_data), plot=True, save=save_figs, case=case,
                            shift_interval=24, 
                            param_variations=param_variations,
                            method='delay', delay_time=4, efficiency=1, approach=approach,
                            cost_dsm_up=cost_dsm_up, cost_dsm_down_shift=cost_dsm_down_shift, 
                            cost_dsm_down_shed=cost_dsm_down_shed,
                            addition=False, fixes=True,
                            recovery_time_shift=None, recovery_time_shed=4, figsize=(15,8), 
                            shed_eligibility=True, shed_time=4, n_yearLimit_shed=6,
                            ActivateDayLimit=True, daily_frequency_shift=dl,
                            introduce_second_dsm_unit=introduce_second_dsm_unit,
                            invest=invest,
                            flex_share_down=flex_share_down,
                            flex_share_up=flex_share_up,
                            ep_costs=0.001,
                            minimum=minimum,
                            maximum=200,
                            max_demand_inv=df_data['demand_el'].max(),
                            max_capacity_down_inv=df_data['demand_el'].max(),
                            max_capacity_up_inv=200-df_data['demand_el'].min())

In [ ]:
show_results_table(approach_dict, save_results, MultiIndex=True)

In [ ]:
results_dict = {}
for k, v in approach_dict.items():
    print(f"approach: {k}\t objective: {v[2]['objective']}")
    results_dict[k] = v[2]['objective']
    
results_df = pd.DataFrame.from_dict(results_dict, orient='index')

In [ ]:
if invest:
    for k, v in approach_dict.items():
        print(f"investment in DR for approach {k}: {v[3]} MW of installed capacity")

## Activation of optional constraint corresponding to Eq. 10 from Zerrahn and Schill (2015a)
* Set parameter addition to True. Contraint limiting overall amount of DSM capacity (sum in both directions) is introduced.
* delay_time is set to three. Otherwise there would be no simulatneous activation in both directions for the example load and generation pattern (case 2b).

### Define settings for analysis of optional overall capacity limit constraint

In [ ]:
case = 'dem_variation_add'

# Data preperation: manipulate demand data
df_data = df_base.copy()
demand = [100] * timesteps

# demand changes
demand[5:6] = [125]
demand[6:7] = [75]

demand[13:14] = [150]
demand[15:16] = [50]

if timesteps > 24:
    demand[25:26] = [25]
    demand[28:29] = [175]

    demand[36:37] = [0]
    demand[40:41] = [200]

df_data['demand_el'] = demand
df_data['Cap_up'] = [100] * timesteps + df_data['Cap_up'] - df_data['demand_el']
df_data['Cap_do'] = [100] * timesteps + df_data['demand_el'] - df_data['Cap_do']

In [ ]:
plot_case(data=df_data, case=case)
plot_case_residual(data=df_data, case=case)

### Analyze overall capacity limit constraint

In [ ]:
# Introduce a dict to store the results of every approach
additions = [False, True]
approach_dict = {}

for add in additions:
    print("-----------------------------------------------------------------------------------------------")
    print("constrain overall DSM capacity {0}".format(addition))
    print("-----------------------------------------------------------------------------------------------")
    for approach in approaches:

        approach_dict[(approach, add)] = \
            start_model(df_data, timesteps=len(df_data), plot=True, save=save_figs, case=case,
                        shift_interval=24, 
                        param_variations='_addition_{0}'.format(addition),
                        method='delay', delay_time=3, efficiency=1, approach=approach,
                        cost_dsm_up=cost_dsm_up, cost_dsm_down_shift=cost_dsm_down_shift, cost_dsm_down_shed=cost_dsm_down_shed,
                        addition=add, recovery_time_shift=None, recovery_time_shed=4, figsize=(15,8),
                        fixes=True,
                        shed_eligibility=True, shed_time=4, n_yearLimit_shed=6,
                        introduce_second_dsm_unit=introduce_second_dsm_unit,
                        invest=invest,
                        flex_share_down=flex_share_down,
                        flex_share_up=flex_share_up,
                        ep_costs=0.001,
                        minimum=minimum,
                        maximum=200,
                        max_demand_inv=df_data['demand_el'].max(),
                        max_capacity_down_inv=df_data['demand_el'].max(),
                        max_capacity_up_inv=200-df_data['demand_el'].min())

In [ ]:
show_results_table(approach_dict, save_results, MultiIndex=True,
                  param_name="add_constr")

In [ ]:
results_dict = {}
for k, v in approach_dict.items():
    print(f"approach: {k}\t objective: {v[2]['objective']}")
    results_dict[k] = v[2]['objective']
    
results_df = pd.DataFrame.from_dict(results_dict, orient='index')

In [ ]:
if invest:
    for k, v in approach_dict.items():
        print(f"investment in DR for approach {k}: {v[3]} MW of installed capacity")

## Variations of parameters for load shedding

In [ ]:
case = 'param_variation_shed'

# Data preperation: manipulate demand data
df_data = df_base.copy()
demand = [100] * timesteps

# demand changes
demand[10:24] = [200] * 14

df_data['demand_el'] = demand
df_data['Cap_up'] = [100] * timesteps + df_data['Cap_up'] - df_data['demand_el']
df_data['Cap_do'] = [100] * timesteps + df_data['demand_el'] - df_data['Cap_do']

In [ ]:
plot_case(data=df_data, case=case)
plot_case_residual(data=df_data, case=case)

### Variations in shed time

In [ ]:
# Introduce a dict to store the results of every approach
shed_times = [1, 2, 3, 4, 5]
approach_dict = {}

for st in shed_times:
    print("-----------------------------------------------------------------------------------------------")
    print("shed time {:d}".format(st))
    print("-----------------------------------------------------------------------------------------------")
    for approach in approaches:

        approach_dict[(approach, st)] = start_model(df_data, timesteps=len(df_data), plot=True, save=save_figs, case=case, 
                                          shift_interval=24, delay_time=4, efficiency=1, approach=approach,
                                          cost_dsm_up=cost_dsm_up, cost_dsm_down_shift=cost_dsm_down_shift, 
                                          cost_dsm_down_shed=0.1,
                                          addition=False, recovery_time_shift=None, recovery_time_shed=4, 
                                          figsize=(15,8), 
                                          fixes=True,
                                          shift_eligibility=True,
                                          shed_eligibility=True, shed_time=st, n_yearLimit_shed=6,
                                          introduce_second_dsm_unit=introduce_second_dsm_unit,
                                          invest=invest,
                                          flex_share_down=flex_share_down,
                                          flex_share_up=flex_share_up,
                                          existing=existing,
                                          ep_costs=0.001,
                                          minimum=minimum,
                                          maximum=200,
                                          max_demand_inv=df_data['demand_el'].max(),
                                          max_capacity_down_inv=df_data['demand_el'].max(),
                                          max_capacity_up_inv=200-df_data['demand_el'].min()
                                         )

In [ ]:
show_results_table(approach_dict, save_results, MultiIndex=True,
                  param_name="delay_time")

In [ ]:
results_dict = {}
for k, v in approach_dict.items():
    print(f"approach: {k}\t objective: {v[2]['objective']}")
    results_dict[k] = v[2]['objective']
    
results_df = pd.DataFrame.from_dict(results_dict, orient='index')

In [ ]:
if invest:
    for k, v in approach_dict.items():
        print(f"investment in DR for approach {k}: {v[3]} MW of installed capacity")

### Variations in recovery time for shedding / amount of activations
Interrelation is the same as for maximum amounts of activations and recovery time for shifting

In [ ]:
# Introduce a dict to store the results of every approach
recovery_time = [1, 2, 4, 16, 44]
approach_dict = {}

for rt in recovery_time:
    
    if rt is not None:
        param_variations='_recovery_time_{:d}'.format(rt)
    else:
        param_variations='_no_recovery_time'

    print("-----------------------------------------------------------------------------------------------")
    if not rt is None:
        print("recovery time {:d}; max. activations: {:.0f}; cumulative shift time {:.0f}".format(rt, 
            len(df_data)//(4+rt), len(df_data)//(4+rt) * 2))
    else:
        print("no limit on recovery time")
    print("-----------------------------------------------------------------------------------------------")
    for approach in approaches:            
        
        approach_dict[(approach, rt)] = start_model(df_data, timesteps=len(df_data), plot=True, save=save_figs, case=case, 
                                          shift_interval=24, delay_time=4, efficiency=1, approach=approach,
                                          cost_dsm_up=cost_dsm_up, cost_dsm_down_shift=cost_dsm_down_shift, 
                                          cost_dsm_down_shed=0.1,
                                          addition=False, recovery_time_shift=None, recovery_time_shed=rt, 
                                          figsize=(15,8), 
                                          fixes=True,
                                          shift_eligibility=True,
                                          shed_eligibility=True, shed_time=4,
                                          introduce_second_dsm_unit=introduce_second_dsm_unit,
                                          invest=invest,
                                          flex_share_down=flex_share_down,
                                          flex_share_up=flex_share_up,
                                          existing=existing,
                                          ep_costs=0.001,
                                          minimum=minimum,
                                          maximum=200,
                                          max_demand_inv=df_data['demand_el'].max(),
                                          max_capacity_down_inv=df_data['demand_el'].max(),
                                          max_capacity_up_inv=200-df_data['demand_el'].min()
                                         )

In [ ]:
show_results_table(approach_dict, save_results, MultiIndex=True,
                  param_name="delay_time")

In [ ]:
results_dict = {}
for k, v in approach_dict.items():
    print(f"approach: {k}\t objective: {v[2]['objective']}")
    results_dict[k] = v[2]['objective']
    
results_df = pd.DataFrame.from_dict(results_dict, orient='index')

In [ ]:
if invest:
    for k, v in approach_dict.items():
        print(f"investment in DR for approach {k}: {v[3]} MW of installed capacity")

### Variations in amount of activations (DLR only)

In [ ]:
# Introduce a dict to store the results of every approach
amount = [6, 4, 3, 2, 1]
approach_dict = {}

for a in amount:
    
    param_variations='_amount_{:d}'.format(rt)


    print("-----------------------------------------------------------------------------------------------")
    print("max. activations: {:.0f}".format(a))
    print("-----------------------------------------------------------------------------------------------")
    for approach in approaches:
        
        if approach == "DLR":
        
            approach_dict[(approach, rt)] = start_model(df_data, timesteps=len(df_data), plot=True, save=save_figs, case=case, 
                                              shift_interval=24, delay_time=4, efficiency=1, approach=approach,
                                              cost_dsm_up=cost_dsm_up, cost_dsm_down_shift=cost_dsm_down_shift, 
                                              cost_dsm_down_shed=0.1,
                                              addition=False, recovery_time_shift=None, recovery_time_shed=4, 
                                              figsize=(15,8), 
                                              fixes=True,
                                              shift_eligibility=True,
                                              shed_eligibility=True, shed_time=4, n_yearLimit_shed=a,
                                              introduce_second_dsm_unit=introduce_second_dsm_unit,
                                              invest=invest,
                                              flex_share_down=flex_share_down,
                                              flex_share_up=flex_share_up,
                                              existing=existing,
                                              ep_costs=0.001,
                                              minimum=minimum,
                                              maximum=200,
                                              max_demand_inv=df_data['demand_el'].max(),
                                              max_capacity_down_inv=df_data['demand_el'].max(),
                                              max_capacity_up_inv=200-df_data['demand_el'].min()
                                             )

In [ ]:
show_results_table(approach_dict, save_results, MultiIndex=True,
                  param_name="delay_time")

In [ ]:
results_dict = {}
for k, v in approach_dict.items():
    print(f"approach: {k}\t objective: {v[2]['objective']}")
    results_dict[k] = v[2]['objective']
    
results_df = pd.DataFrame.from_dict(results_dict, orient='index')

In [ ]:
if invest:
    for k, v in approach_dict.items():
        print(f"investment in DR for approach {k}: {v[3]} MW of installed capacity")

# Bibliography

* Gils, Hans Christian (2015): Balancing of Intermittent Renewable Power Generation by Demand Response and Thermal Energy Storage, Stuttgart, http://dx.doi.org/10.18419/opus-6888, accessed 16.08.2019, pp. 67-70.
* Kochems, Johannes (2020): Demand response potentials for Germany: potential clustering and comparison of modeling approaches, presentation at the INREC conference 2020, https://github.com/jokochems/DR_modeling_oemof/blob/master/Kochems_Demand_Response_INREC.pdf, accessed 20.03.2021.
* Zerrahn, Alexander and Schill, Wolf-Peter (2015a): On the representation of demand-side management in power system models, in: Energy (84), pp. 840-845, [10.1016/j.energy.2015.03.037](http://dx.doi.org/10.1016/j.energy.2015.03.037), accessed 16.08.2019, pp. 842-843.
* Zerrahn, Alexander and Schill, Wolf-Peter (2015b): A Greenfield Model to Evaluate Long-Run Power Storage Requirements for High Shares of Renewables, DIW Discussion Papers, No. 1457.